In [ ]:
# ============================================================================
# CELL 0: SETUP AND CONFIGURATION
# ============================================================================
# Purpose: Initialize environment, set reproducibility seeds, define constants
# Key components:
#   - Library imports
#   - Random seed configuration for reproducibility
#   - Dataset paths and feature definitions
#   - Device selection (GPU/CPU)
#   - Data preparation function
# ============================================================================

from pathlib import Path
import json
import time
import warnings
from collections import defaultdict, Counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
from sklearn.model_selection import KFold, train_test_split
from scipy.stats import chi2, chi2_contingency, pearsonr
from torch_lr_finder import LRFinder

warnings.filterwarnings('ignore')

RANDOM_SEED = 42
FEATURE_COLUMNS = ["Depression", "Anxiety", "Stress", "Burnout"]
DATASETS = {
    "D1-Swiss": Path("D1_Swiss_processed.csv"),
    "D2-Cultural": Path("D2_Cultural_processed.csv"),
    "D3-Academic": Path("D3_Academic_processed.csv"),
    "D4-Tech": Path("D4_Tech_processed.csv"),
}
# Optimal bin numbers for stratified splitting (from grid search)
STRATIFICATION_BINS = {
    "D1-Swiss": 2,
    "D2-Cultural": 2,
    "D3-Academic": 4,
    "D4-Tech": 2,
}
PIPELINE_RESULTS = {}

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

def prepare_dataset(dataset_name: str):
    if dataset_name not in DATASETS:
        raise ValueError(f"Unknown dataset '{dataset_name}'. Available: {list(DATASETS.keys())}")

    dataset_path = DATASETS[dataset_name]
    print(f"\n=== Loading {dataset_name} dataset ===")
    print(f"File: {dataset_path}")
    print(f"Features: {FEATURE_COLUMNS}")

    df = pd.read_csv(dataset_path)
    
    # Check if required columns exist
    missing_cols = [col for col in FEATURE_COLUMNS if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns in {dataset_path}: {missing_cols}")
    
    # Check if dataset is empty
    if len(df) == 0:
        raise ValueError(f"Dataset {dataset_path} is empty.")
    
    feature_matrix = df[FEATURE_COLUMNS].values
    print(f"Dataset: {feature_matrix.shape[0]} samples, {feature_matrix.shape[1]} features")
    
    # Check if we have enough samples for train/test split
    if len(feature_matrix) < 10:
        raise ValueError(f"Dataset too small ({len(feature_matrix)} samples). Need at least 10 samples.")

    # ========================================================================
    # STRATIFIED SPLIT: Use binned features for stratification
    # ========================================================================
    # Create bins for each continuous feature to enable stratification
    # This ensures train/test sets have similar distributions
    # ========================================================================
    try:
        n_bins = STRATIFICATION_BINS.get(dataset_name, 2)  # Default to 2 bins if not specified
        
        # Create bins for each feature (using quantiles)
        df_binned = df.copy()
        for col in FEATURE_COLUMNS:
            # Use quantiles to create bins, handle duplicates
            df_binned[f'{col}_bin'] = pd.qcut(df[col], q=n_bins, labels=False, duplicates='drop')
        
        # Create stratification label (combination of all feature bins)
        df_binned['stratify_label'] = df_binned[[f'{col}_bin' for col in FEATURE_COLUMNS]].apply(
            lambda x: '_'.join(x.astype(str)), axis=1
        )
        
        # Check if we have enough samples per stratum for stratification
        stratum_counts = df_binned['stratify_label'].value_counts()
        min_stratum_size = stratum_counts.min()
        
        if min_stratum_size < 2:
            raise ValueError(f"Some strata have < 2 samples (min = {min_stratum_size}), cannot stratify")
        
        # Perform stratified split
        train_val_data, test_data = train_test_split(
            feature_matrix, 
            test_size=0.2, 
            random_state=RANDOM_SEED,
            stratify=df_binned['stratify_label']
        )
        print(f"✓ Using STRATIFIED split with {n_bins} bins per feature (ensures balanced train/test distributions)")
        
    except (ValueError, KeyError) as e:
        # Fallback to regular split if stratification fails
        print(f"⚠ Stratification failed ({str(e)}), using regular split")
        train_val_data, test_data = train_test_split(
            feature_matrix, 
            test_size=0.2, 
            random_state=RANDOM_SEED
        )

    train_val_tensor = torch.tensor(train_val_data, dtype=torch.float32)
    test_tensor = torch.tensor(test_data, dtype=torch.float32)
    kfold = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

    print(f"Train+Val: {train_val_tensor.shape[0]} samples (80%)")
    print(f"Test: {test_tensor.shape[0]} samples (20%)")
    print(f"K-Fold: 10 folds, ~{train_val_tensor.shape[0]//10} samples per fold\n")

    return (
        df,
        feature_matrix,
        train_val_data,
        test_data,
        train_val_tensor,
        test_tensor,
        kfold,
        dataset_path,
    )

INPUT_DIM = len(FEATURE_COLUMNS)



In [ ]:
# ============================================================================
# CELL 1: AUTOENCODER ARCHITECTURE DEFINITION
# ============================================================================
# Purpose: Define the neural network that compresses 4D symptoms into
#          lower-dimensional latent space for clustering
# Architecture: Symmetric encoder-decoder with configurable dimensions
# ============================================================================

class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, activation_function):
        super(Autoencoder, self).__init__()

        #Encoder - input -> hidden -> latent
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            activation_function(),
            nn.Linear(hidden_dim, latent_dim)
        )
        

        #Decoder - latent -> hidden -> output
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            activation_function(),
            nn.Linear(hidden_dim, input_dim)
        )
        
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
# ============================================================================
# CELL 2: MAIN PIPELINE - Hyperparameter Tuning and Profile Extraction
# ============================================================================
# Purpose: Two-stage hyperparameter optimization followed by profile extraction
# Stage 1: Architecture tuning (hidden_size, latent_dim, activation, optimizer)
# Stage 2: Learning parameter tuning (batch_size, weight_decay, momentum)
# Final: Train on all data, extract profiles, evaluate on test set
# ============================================================================

def run_autoencoder_pipeline(dataset_name: str, force_latent_dim: int = None, force_k: int = None, narrowed_params: dict = None, n_folds: int = 5):
    (
        all_data_df,
        all_data,
        train_val_data,
        test_data,
        train_val_tensor,
        test_tensor,
        kfold,
        dataset_path,
    ) = prepare_dataset(dataset_name)
    
    # ------------------------------------------------------------------------
    # STAGE 1: Architecture Parameter Tuning
    # ------------------------------------------------------------------------
    # Grid search over architecture parameters with 10-fold cross-validation
    # Selection criterion: Consensus voting on clustering quality metrics
    # ------------------------------------------------------------------------
    print("STAGE 1: Architecture Parameters Tuning (K-Fold CV)")
    
    # Require narrowed params from Optuna scout phase
    if not narrowed_params:
        raise ValueError(
            "narrowed_params is required! Run the Optuna scout phase first to generate narrowed parameters.\n"
            "Usage: run_scout_phase() → extract_narrowed_ranges() → run_autoencoder_pipeline(narrowed_params=...)"
        )
    
    print("Using NARROWED parameter ranges from Optuna scout phase")
    hidden_sizes = narrowed_params.get("hidden_sizes", [2, 3, 4, 5, 6, 8, 10])
    latent_dims = narrowed_params.get("latent_dims", [2, 3, 4])
    
    all_activations = {"ReLU": nn.ReLU, "Tanh": nn.Tanh, "Sigmoid": nn.Sigmoid}
    activations = {name: all_activations[name] for name in narrowed_params.get("activations", ["ReLU", "Tanh", "Sigmoid"])}
    
    all_optimizers = {"Adam": optim.Adam, "SGD": optim.SGD}
    optimizers = {name: all_optimizers[name] for name in narrowed_params.get("optimizers", ["Adam", "SGD"])}
    
    epochs_list = narrowed_params.get("epochs", [50, 75, 100])
    fixed_lr = 1e-3

    criterion = nn.MSELoss()
    results_stage1 = defaultdict(list)

    total_experiments = len(hidden_sizes) * len(latent_dims) * len(activations) * len(optimizers) * len(epochs_list) * n_folds
    experiment_count = 0

    print(f"Testing: hidden_size, latent_dim, activation, optimizer, epochs")
    print(f"Fixed: learning_rate = {fixed_lr}")
    print(f"\nGrid sizes:")
    print(f"  hidden_size: {len(hidden_sizes)} values {hidden_sizes}")
    print(f"  latent_dim: {len(latent_dims)} values {latent_dims}")
    print(f"  activation: {len(activations)} values {list(activations.keys())}")
    print(f"  optimizer: {len(optimizers)} values {list(optimizers.keys())}")
    print(f"  epochs: {len(epochs_list)} values {epochs_list}")
    print(f"  CV folds: {n_folds}")
    print(f"Total: {total_experiments} experiments ({total_experiments//n_folds} configs × {n_folds} folds)\n")

    start_stage1 = time.time()

    for hidden_size in hidden_sizes:
        for latent_dim in latent_dims:
            for act_name, act_fn in activations.items():
                for opt_name, opt_class in optimizers.items():
                    for num_epochs in epochs_list:
                        for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(train_val_tensor)):
                            experiment_count += 1
                            if experiment_count % 50 == 0 or experiment_count == total_experiments:
                                elapsed = time.time() - start_stage1
                                print(f"  [{experiment_count}/{total_experiments}] {100*experiment_count/total_experiments:.1f}% - {elapsed/60:.1f}min")
                        
                            fold_seed = 42 + fold_idx
                            torch.manual_seed(fold_seed)
                            np.random.seed(fold_seed)
                        
                            train_fold = train_val_tensor[train_idx]
                            val_fold = train_val_tensor[val_idx]
                        
                            train_dataset = TensorDataset(train_fold.cpu())
                            val_dataset = TensorDataset(val_fold.cpu())
                            train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
                            val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
                        
                            model = Autoencoder(INPUT_DIM, hidden_size, latent_dim, act_fn).to(device)
                        
                            if opt_name == 'Adam':
                                optimizer = opt_class(model.parameters(), lr=fixed_lr)
                            else:
                                optimizer = opt_class(model.parameters(), lr=fixed_lr, momentum=0.9)
                        
                            train_losses, val_losses, optimal_k, best_sil_score, latent_vectors, validation_metrics = \
                                train_and_validate_model(model, train_loader, val_loader, optimizer, 
                                                        criterion, num_epochs, device)
                        
                            # Safely get index for optimal_k
                            if optimal_k in validation_metrics['k_values']:
                                optimal_k_idx = validation_metrics['k_values'].index(optimal_k)
                                best_ch_score = validation_metrics['calinski_harabasz_scores'][optimal_k_idx]
                                best_db_score = validation_metrics['davies_bouldin_scores'][optimal_k_idx]
                            else:
                                # Fallback: use first available K
                                optimal_k_idx = 0
                                best_ch_score = validation_metrics['calinski_harabasz_scores'][0] if len(validation_metrics['calinski_harabasz_scores']) > 0 else 0
                                best_db_score = validation_metrics['davies_bouldin_scores'][0] if len(validation_metrics['davies_bouldin_scores']) > 0 else float('inf')
                        
                            results_stage1['hidden_size'].append(hidden_size)
                            results_stage1['latent_dim'].append(latent_dim)
                            results_stage1['activation'].append(act_name)
                            results_stage1['optimizer'].append(opt_name)
                            results_stage1['epochs'].append(num_epochs)
                            results_stage1['fold'].append(fold_idx)
                            results_stage1['optimal_k'].append(optimal_k)
                            results_stage1['silhouette_score'].append(best_sil_score)
                            results_stage1['calinski_harabasz_score'].append(best_ch_score)
                            results_stage1['davies_bouldin_score'].append(best_db_score)
                            results_stage1['reconstruction_loss'].append(val_losses[-1])
                            results_stage1['consensus_reached'].append(validation_metrics['consensus_reached'])

    print(f"\n✓ Stage 1 completed in {(time.time()-start_stage1)/60:.2f} minutes\n")

    # ------------------------------------------------------------------------
    # STAGE 1: Results Aggregation and Best Configuration Selection
    # ------------------------------------------------------------------------
    # Aggregate results across folds, rank by clustering metrics,
    # use consensus voting to select best architecture
    # ------------------------------------------------------------------------
    print("STAGE 1: Results Aggregation")
    print("="*70)

    stage1_df = pd.DataFrame(results_stage1)
    
    # Check if we have any results
    if len(stage1_df) == 0:
        raise ValueError("Stage 1 produced no results. Check training loop.")

    aggregated_stage1 = stage1_df.groupby(['hidden_size', 'latent_dim', 'activation', 'optimizer', 'epochs']).agg({
        'silhouette_score': ['mean', 'std'],
        'calinski_harabasz_score': ['mean', 'std'],
        'davies_bouldin_score': ['mean', 'std'],
        'reconstruction_loss': ['mean', 'std'],
        'optimal_k': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0]
    }).reset_index()

    aggregated_stage1.columns = ['hidden_size', 'latent_dim', 'activation', 'optimizer', 'epochs',
                                  'mean_silhouette', 'std_silhouette',
                                  'mean_ch', 'std_ch',
                                  'mean_db', 'std_db',
                                  'mean_recon_loss', 'std_recon_loss',
                                  'most_common_k']

    # Create config_id BEFORE filtering
    aggregated_stage1['config_id'] = aggregated_stage1.apply(
        lambda row: f"{row['hidden_size']}_{row['latent_dim']}_{row['activation']}_{row['optimizer']}_{row['epochs']}", 
        axis=1
    )
    
    aggregated_stage1['rank_silhouette'] = aggregated_stage1['mean_silhouette'].rank(ascending=False, method='min')
    aggregated_stage1['rank_ch'] = aggregated_stage1['mean_ch'].rank(ascending=False, method='min')
    aggregated_stage1['rank_db'] = aggregated_stage1['mean_db'].rank(ascending=True, method='min')

    top_by_sil = aggregated_stage1.loc[aggregated_stage1['rank_silhouette'] == 1]
    top_by_ch = aggregated_stage1.loc[aggregated_stage1['rank_ch'] == 1]
    top_by_db = aggregated_stage1.loc[aggregated_stage1['rank_db'] == 1]

    votes = []
    if len(top_by_sil) > 0:
        votes.extend(top_by_sil['config_id'].tolist())
    if len(top_by_ch) > 0:
        votes.extend(top_by_ch['config_id'].tolist())
    if len(top_by_db) > 0:
        votes.extend(top_by_db['config_id'].tolist())

    # Check if aggregated_stage1 is empty
    if len(aggregated_stage1) == 0:
        raise ValueError("Stage 1 aggregation produced no results. Check groupby operation.")
    
    vote_counts = Counter(votes)
    if len(vote_counts) > 0:
        most_voted_config, vote_count = vote_counts.most_common(1)[0]
    
        if vote_count >= 2:
            matching_configs = aggregated_stage1[aggregated_stage1['config_id'] == most_voted_config]
            if len(matching_configs) == 0:
                raise ValueError(f"Config {most_voted_config} not found in aggregated results.")
            best_config_stage1 = matching_configs.iloc[0]
            consensus_status = f"Consensus: {vote_count} metrics agree"
        else:
            top_sil = aggregated_stage1.loc[aggregated_stage1['rank_silhouette'] == 1]
            if len(top_sil) == 0:
                raise ValueError("No top silhouette config found.")
            best_config_stage1 = top_sil.iloc[0]
            consensus_status = f"No consensus. Using Silhouette"
    else:
        aggregated_stage1_sorted = aggregated_stage1.sort_values('mean_silhouette', ascending=False)
        if len(aggregated_stage1_sorted) == 0:
            raise ValueError("No configurations to select from.")
        best_config_stage1 = aggregated_stage1_sorted.iloc[0]
        consensus_status = "Using Silhouette (fallback)"

    best_hidden_size = int(best_config_stage1['hidden_size'])
    best_latent_dim = int(best_config_stage1['latent_dim'])
    best_activation_name = best_config_stage1['activation']
    best_optimizer_name = best_config_stage1['optimizer']
    best_epochs = int(best_config_stage1['epochs'])
    best_k = force_k if force_k is not None else int(best_config_stage1['most_common_k'])

    print(f"Top 5 configs (by mean silhouette score):\n")
    aggregated_stage1_sorted = aggregated_stage1.sort_values('mean_silhouette', ascending=False)
    print(aggregated_stage1_sorted.head(5)[['hidden_size', 'latent_dim', 'activation', 'optimizer', 
                                            'epochs', 'mean_silhouette', 'std_silhouette',
                                            'mean_ch', 'mean_db', 'most_common_k']].to_string(index=False))

    print(f"\n{'='*70}")
    print(f"Best Architecture (Stage 1): {consensus_status}")
    print(f"  hidden_size={best_hidden_size}, latent_dim={best_latent_dim}")
    print(f"  activation={best_activation_name}, optimizer={best_optimizer_name}, epochs={best_epochs}")
    print(f"  Silhouette: {best_config_stage1['mean_silhouette']:.6f} ± {best_config_stage1['std_silhouette']:.6f}")
    print(f"  Calinski-Harabasz: {best_config_stage1['mean_ch']:.6f} ± {best_config_stage1['std_ch']:.6f}")
    print(f"  Davies-Bouldin: {best_config_stage1['mean_db']:.6f} ± {best_config_stage1['std_db']:.6f}")
    print(f"  Reconstruction loss: {best_config_stage1['mean_recon_loss']:.6f} ± {best_config_stage1['std_recon_loss']:.6f}")
    print(f"  Most common optimal K: {best_k}")
    print(f"{'='*70}\n")

    # ------------------------------------------------------------------------
    # STAGE 2: Learning Parameter Optimization
    # ------------------------------------------------------------------------
    # Fine-tune training parameters using best architecture from Stage 1
    # Learning rate determined via LR Range Test
    # ------------------------------------------------------------------------
    print("STAGE 2: Learning Parameter Optimization (K-Fold CV)")
    print("="*70)
    print(f"Using best architecture from Stage 1:")
    print(f"  hidden={best_hidden_size}, latent={best_latent_dim}, activation={best_activation_name}, optimizer={best_optimizer_name}")

    activation_map = {'ReLU': nn.ReLU, 'Tanh': nn.Tanh, 'Sigmoid': nn.Sigmoid}
    best_activation_fn = activation_map[best_activation_name]

    # ------------------------------------------------------------------------
    # LR Range Test: Find Optimal Learning Rate
    # ------------------------------------------------------------------------
    # Test learning rates from 1e-7 to 10 to find optimal value
    # Uses subset of training data for efficiency
    # ------------------------------------------------------------------------
    print("\n" + "="*70)
    print("LR RANGE TEST (Using Best Architecture from Stage 1)")
    print("="*70)

    # Create DataLoader that returns (input, target) pairs for LR finder
    # For autoencoder, target = input (reconstruction task)
    train_subset_data = train_val_tensor[:500]
    train_subset = TensorDataset(train_subset_data, train_subset_data)  # (input, target) where target=input
    train_loader_lr = DataLoader(train_subset, batch_size=64, shuffle=True)
    criterion = nn.MSELoss()

    print(f"\nLR Range Test: {best_optimizer_name} optimizer")
    model_lr = Autoencoder(INPUT_DIM, best_hidden_size, best_latent_dim, best_activation_fn).to(device)

    if best_optimizer_name == 'Adam':
        optimizer_lr = optim.Adam(model_lr.parameters(), lr=1e-7)
    else:
        optimizer_lr = optim.SGD(model_lr.parameters(), lr=1e-7, momentum=0.9)

    lr_finder = LRFinder(model_lr, optimizer_lr, criterion, device=device)
    lr_finder.range_test(train_loader_lr, end_lr=10, num_iter=100, step_mode="exp")
    lr_finder.plot()
    plt.title(f'{best_optimizer_name} LR Range Test (Best Architecture)', fontsize=14, fontweight='bold')
    plt.show()

    history = lr_finder.history
    lrs = np.array(history['lr'])
    losses = np.array(history['loss'])
    loss_diffs = np.diff(losses)
    descending = np.where(loss_diffs < 0)[0]

    if len(descending) > 0:
        mid = (descending[0] + descending[-1]) // 2
        optimal_lr = lrs[mid]
    else:
        optimal_lr = lrs[np.argmin(losses)]

    print(f"Optimal LR from range test: {optimal_lr:.2e}\n")

    lr_finder.reset()

    batch_sizes = [32, 64, 128]
    weight_decays = [0, 1e-4, 1e-3]

    if best_optimizer_name == 'SGD':
        momentum_values = [0.5, 0.9, 0.95]
        print(f"Testing: batch_size, weight_decay, momentum")
        print(f"Fixed: learning_rate = {optimal_lr:.2e}")
        total_experiments = len(batch_sizes) * len(weight_decays) * len(momentum_values) * n_folds
    else:
        momentum_values = [None]
        print(f"Testing: batch_size, weight_decay")
        print(f"Fixed: learning_rate = {optimal_lr:.2e}")
        total_experiments = len(batch_sizes) * len(weight_decays) * n_folds

    results_stage2 = defaultdict(list)
    experiment_count = 0

    print(f"\nGrid sizes:")
    print(f"  learning_rate: 1 value (optimal from LR range test)")
    print(f"  batch_size: {len(batch_sizes)} values {batch_sizes}")
    print(f"  weight_decay: {len(weight_decays)} values {weight_decays}")
    if best_optimizer_name == 'SGD':
        print(f"  momentum: {len(momentum_values)} values {momentum_values}")
    print(f"  CV folds: {n_folds}")
    print(f"Total: {total_experiments} experiments ({total_experiments//n_folds} configs × {n_folds} folds)\n")

    start_stage2 = time.time()

    for batch_size in batch_sizes:
        for weight_decay in weight_decays:
            for momentum in momentum_values:
                for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(train_val_tensor)):
                    experiment_count += 1
                    if experiment_count % 50 == 0 or experiment_count == total_experiments:
                        elapsed = time.time() - start_stage2
                        print(f"  [{experiment_count}/{total_experiments}] {100*experiment_count/total_experiments:.1f}% - {elapsed/60:.1f}min")
                
                    fold_seed = 42 + fold_idx
                    torch.manual_seed(fold_seed)
                    np.random.seed(fold_seed)
                
                    train_fold = train_val_tensor[train_idx]
                    val_fold = train_val_tensor[val_idx]
                
                    train_dataset = TensorDataset(train_fold.cpu())
                    val_dataset = TensorDataset(val_fold.cpu())
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                
                    model = Autoencoder(INPUT_DIM, best_hidden_size, best_latent_dim, best_activation_fn).to(device)
                
                    if best_optimizer_name == 'SGD':
                        optimizer = optim.SGD(model.parameters(), lr=optimal_lr, momentum=momentum, weight_decay=weight_decay)
                    else:
                        optimizer = optim.Adam(model.parameters(), lr=optimal_lr, weight_decay=weight_decay)
                
                    train_losses, val_losses, optimal_k, best_sil_score, latent_vectors, validation_metrics = \
                        train_and_validate_model(model, train_loader, val_loader, optimizer, 
                                                criterion, best_epochs, device)
                
                    # Safely get index for optimal_k
                    if optimal_k in validation_metrics['k_values']:
                        optimal_k_idx = validation_metrics['k_values'].index(optimal_k)
                        best_ch_score = validation_metrics['calinski_harabasz_scores'][optimal_k_idx]
                        best_db_score = validation_metrics['davies_bouldin_scores'][optimal_k_idx]
                    else:
                        # Fallback: use first available K
                        optimal_k_idx = 0
                        best_ch_score = validation_metrics['calinski_harabasz_scores'][0] if len(validation_metrics['calinski_harabasz_scores']) > 0 else 0
                        best_db_score = validation_metrics['davies_bouldin_scores'][0] if len(validation_metrics['davies_bouldin_scores']) > 0 else float('inf')
                
                    results_stage2['learning_rate'].append(optimal_lr)
                    results_stage2['batch_size'].append(batch_size)
                    results_stage2['weight_decay'].append(weight_decay)
                    if best_optimizer_name == 'SGD':
                        results_stage2['momentum'].append(momentum)
                    results_stage2['fold'].append(fold_idx)
                    results_stage2['silhouette_score'].append(best_sil_score)
                    results_stage2['calinski_harabasz_score'].append(best_ch_score)
                    results_stage2['davies_bouldin_score'].append(best_db_score)
                    results_stage2['reconstruction_loss'].append(val_losses[-1])

    print(f"\n✓ Stage 2 completed in {(time.time()-start_stage2)/60:.2f} minutes\n")

    # ------------------------------------------------------------------------
    # STAGE 2: Results Aggregation and Best Configuration Selection
    # ------------------------------------------------------------------------
    # Aggregate results across folds, select best learning parameters
    # ------------------------------------------------------------------------
    print("STAGE 2: Results Aggregation")
    print("="*70)

    stage2_df = pd.DataFrame(results_stage2)
    
    # Check if we have any results
    if len(stage2_df) == 0:
        raise ValueError("Stage 2 produced no results. Check training loop.")

    if 'momentum' in results_stage2:
        groupby_cols = ['learning_rate', 'batch_size', 'weight_decay', 'momentum']
    else:
        groupby_cols = ['learning_rate', 'batch_size', 'weight_decay']

    aggregated_stage2 = stage2_df.groupby(groupby_cols).agg({
        'silhouette_score': ['mean', 'std'],
        'calinski_harabasz_score': ['mean', 'std'],
        'davies_bouldin_score': ['mean', 'std'],
        'reconstruction_loss': ['mean', 'std']
    }).reset_index()

    col_names = groupby_cols + ['mean_silhouette', 'std_silhouette', 'mean_ch', 'std_ch', 
                                'mean_db', 'std_db', 'mean_recon_loss', 'std_recon_loss']
    aggregated_stage2.columns = col_names

    # Create config_id BEFORE filtering
    # Check if aggregated_stage2 is empty
    if len(aggregated_stage2) == 0:
        raise ValueError("Stage 2 aggregation produced no results. Check groupby operation.")
    
    aggregated_stage2['config_id'] = aggregated_stage2.apply(
        lambda row: f"{row['learning_rate']:.2e}_{row['batch_size']}_{row['weight_decay']:.2e}" + 
                    (f"_{row['momentum']}" if 'momentum' in row.index else ""), 
        axis=1
    )
    
    aggregated_stage2['rank_silhouette'] = aggregated_stage2['mean_silhouette'].rank(ascending=False, method='min')
    aggregated_stage2['rank_ch'] = aggregated_stage2['mean_ch'].rank(ascending=False, method='min')
    aggregated_stage2['rank_db'] = aggregated_stage2['mean_db'].rank(ascending=True, method='min')

    top_by_sil = aggregated_stage2.loc[aggregated_stage2['rank_silhouette'] == 1]
    top_by_ch = aggregated_stage2.loc[aggregated_stage2['rank_ch'] == 1]
    top_by_db = aggregated_stage2.loc[aggregated_stage2['rank_db'] == 1]

    votes = []
    if len(top_by_sil) > 0:
        votes.extend(top_by_sil['config_id'].tolist())
    if len(top_by_ch) > 0:
        votes.extend(top_by_ch['config_id'].tolist())
    if len(top_by_db) > 0:
        votes.extend(top_by_db['config_id'].tolist())

    vote_counts = Counter(votes)
    if len(vote_counts) > 0:
        most_voted_config, vote_count = vote_counts.most_common(1)[0]
    
        if vote_count >= 2:
            matching_configs = aggregated_stage2[aggregated_stage2['config_id'] == most_voted_config]
            if len(matching_configs) == 0:
                raise ValueError(f"Config {most_voted_config} not found in aggregated results.")
            best_config_stage2 = matching_configs.iloc[0]
            consensus_status = f"Consensus: {vote_count} metrics agree"
        else:
            top_sil = aggregated_stage2.loc[aggregated_stage2['rank_silhouette'] == 1]
            if len(top_sil) == 0:
                raise ValueError("No top silhouette config found.")
            best_config_stage2 = top_sil.iloc[0]
            consensus_status = f"No consensus. Using Silhouette"
    else:
        aggregated_stage2_sorted = aggregated_stage2.sort_values('mean_silhouette', ascending=False)
        if len(aggregated_stage2_sorted) == 0:
            raise ValueError("No configurations to select from.")
        best_config_stage2 = aggregated_stage2_sorted.iloc[0]
        consensus_status = "Using Silhouette (fallback)"

    best_learning_rate = best_config_stage2['learning_rate']
    best_batch_size = int(best_config_stage2['batch_size'])
    best_weight_decay = best_config_stage2['weight_decay']
    best_momentum = best_config_stage2.get('momentum', None)

    print(f"Top 5 configs (by mean silhouette score):\n")
    aggregated_stage2_sorted = aggregated_stage2.sort_values('mean_silhouette', ascending=False)
    print(aggregated_stage2_sorted.head(5)[groupby_cols + ['mean_silhouette', 'std_silhouette', 
                                                            'mean_ch', 'mean_db']].to_string(index=False))

    print(f"\n{'='*70}")
    print(f"Best Overall Configuration: {consensus_status}")
    print(f"{'='*70}")
    print(f"Architecture (from Stage 1):")
    print(f"  hidden_size={best_hidden_size}, latent_dim={best_latent_dim}")
    print(f"  activation={best_activation_name}, optimizer={best_optimizer_name}, epochs={best_epochs}")
    print(f"\nLearning Parameters (from Stage 2):")
    print(f"  learning_rate={best_learning_rate:.2e}")
    print(f"  batch_size={best_batch_size}")
    print(f"  weight_decay={best_weight_decay:.2e}", end="")
    if best_momentum is not None:
        print(f", momentum={best_momentum}")
    else:
        print()
    print(f"\nPerformance:")
    print(f"  Silhouette: {best_config_stage2['mean_silhouette']:.6f} ± {best_config_stage2['std_silhouette']:.6f}")
    print(f"  Calinski-Harabasz: {best_config_stage2['mean_ch']:.6f} ± {best_config_stage2['std_ch']:.6f}")
    print(f"  Davies-Bouldin: {best_config_stage2['mean_db']:.6f} ± {best_config_stage2['std_db']:.6f}")
    print(f"  Reconstruction loss: {best_config_stage2['mean_recon_loss']:.6f} ± {best_config_stage2['std_recon_loss']:.6f}")
    print(f"  Optimal K: {best_k}")
    print(f"{'='*70}\n")
    
    # ------------------------------------------------------------------------
    # FINAL MODEL TRAINING AND LATENT PROFILE EXTRACTION
    # ------------------------------------------------------------------------
    # Train final model on all train+val data with best hyperparameters
    # Extract latent vectors and perform K-means clustering
    # ------------------------------------------------------------------------
    print("Final Model Training and Latent Profile Extraction")
    print("="*70)
    print(f"Training final model on all {dataset_name} train+val data with best hyperparameters:")
    print(f"  Architecture: hidden={best_hidden_size}, latent={best_latent_dim}, activation={best_activation_name}")
    print(f"  Optimizer: {best_optimizer_name}, lr={best_learning_rate:.2e}, batch_size={best_batch_size}")
    print(f"  Epochs: {best_epochs}, Optimal K: {best_k}")
    print("="*70)

    torch.manual_seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)

    final_dataset = TensorDataset(train_val_tensor.cpu())
    final_loader = DataLoader(final_dataset, batch_size=best_batch_size, shuffle=True)

    activation_map = {'ReLU': nn.ReLU, 'Tanh': nn.Tanh, 'Sigmoid': nn.Sigmoid}
    best_activation_fn = activation_map[best_activation_name]
    final_model = Autoencoder(INPUT_DIM, best_hidden_size, best_latent_dim, best_activation_fn).to(device)

    if best_optimizer_name == 'SGD':
        final_optimizer = optim.SGD(final_model.parameters(), lr=best_learning_rate, 
                                    momentum=best_momentum if best_momentum is not None else 0.9, 
                                    weight_decay=best_weight_decay)
    else:
        final_optimizer = optim.Adam(final_model.parameters(), lr=best_learning_rate, 
                                    weight_decay=best_weight_decay)

    criterion = nn.MSELoss()

    print(f"\nTraining final model...")
    final_model.train()
    final_losses = []

    for epoch in range(best_epochs):
        epoch_loss = 0.0
        for batch_data in final_loader:
            batch_data = batch_data[0].to(device)
            final_optimizer.zero_grad()
            reconstructed = final_model(batch_data)
            loss = criterion(reconstructed, batch_data)
            loss.backward()
            final_optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(final_loader)
        final_losses.append(avg_loss)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{best_epochs}, Loss: {avg_loss:.4f}")

    print(f"\nFinal model training complete. Saving model...")
    final_model.eval()
    all_latent_vectors_batches = []

    with torch.no_grad():
        for batch_data in final_loader:
            data = batch_data[0].to(device)
            latent = final_model.encoder(data)
            all_latent_vectors_batches.append(latent.cpu())
    latent_vectors_all = np.vstack(all_latent_vectors_batches)
    print(f"Latent vectors shape: {latent_vectors_all.shape}")

    # ------------------------------------------------------------------------
    # K-Means Clustering: Extract Mental Health Profiles
    # ------------------------------------------------------------------------
    # Cluster latent vectors to identify distinct mental health profiles
    # ------------------------------------------------------------------------
    print(f"Running K-means clustering with optimal k... {best_k}")
    final_kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_SEED, n_init=10)
    cluster_labels_all = final_kmeans.fit_predict(latent_vectors_all)
    cluster_centroids = final_kmeans.cluster_centers_

    print(f"Cluster Assignments: {cluster_labels_all}")
    print(f"Cluster Centroids: {cluster_centroids.shape}")

    # Check if we have at least 2 clusters (required for silhouette score)
    n_unique_clusters = len(np.unique(cluster_labels_all))
    if n_unique_clusters < 2:
        print(f"⚠️ ERROR: Final clustering produced only {n_unique_clusters} cluster(s). Cannot compute metrics.")
        final_sil_score = -1
        final_ch_score = 0
        final_db_score = float('inf')
    else:
        final_sil_score = silhouette_score(latent_vectors_all, cluster_labels_all)
        print(f"Final Silhouette Score: {final_sil_score:.4f}")
        final_ch_score = calinski_harabasz_score(latent_vectors_all, cluster_labels_all)
        print(f"Final Calinski-Harabasz Score: {final_ch_score:.4f}")
        final_db_score = davies_bouldin_score(latent_vectors_all, cluster_labels_all)
        print(f"Final Davies-Bouldin Score: {final_db_score:.4f}")

    # ------------------------------------------------------------------------
    # PROFILE CHARACTERISTICS EXTRACTION
    # ------------------------------------------------------------------------
    # Map cluster labels back to original symptom space to interpret profiles
    # Compute mean symptom levels (Depression, Anxiety, Stress, Burnout) per cluster
    # ------------------------------------------------------------------------
    print(f"\nProfile Characteristics (mean feature values per cluster):")
    print("="*70)

    # VERIFICATION: Check that column order matches FEATURE_COLUMNS
    print("Verifying feature column order...")
    sample_0 = train_val_data[0]
    print(f"FEATURE_COLUMNS: {FEATURE_COLUMNS}")
    print(f"First sample values:")
    for i, feature_name in enumerate(FEATURE_COLUMNS):
        print(f"  train_val_data[0, {i}] = {sample_0[i]:.4f} → {feature_name}")
    print("="*70 + "\n")

    profile_summary = []

    for k in range(best_k):
        cluster_mask = cluster_labels_all == k
        # CRITICAL: Use original 4D symptom data, NOT latent vectors as they are more intellgible to understand  
        cluster_data = train_val_data[cluster_mask]  # Original 4D: [Depression, Anxiety, Stress, Burnout]
        cluster_size = np.sum(cluster_mask)
    
        # Method 1: Using array indexing (for verification)
        feature_means_array = cluster_data.mean(axis=0)
    
        # Method 2: Using DataFrame for explicit mapping (SAFER)
        cluster_df = pd.DataFrame(cluster_data, columns=FEATURE_COLUMNS)
        feature_means_dict = cluster_df.mean().to_dict()
    
        # Verify they match
        print(f"Cluster {k} (N={cluster_size}):")
        for i, feature_name in enumerate(FEATURE_COLUMNS):
            array_val = feature_means_array[i]
            dict_val = feature_means_dict[feature_name]
            match = " MATCH" if abs(array_val - dict_val) < 1e-10 else " MISMATCH"
            print(f"  Index {i} ({feature_name}): array[{i}]={array_val:.6f}, dict['{feature_name}']={dict_val:.6f} {match}")
    
        # Use dictionary approach (explicit, no index guessing)
        profile_summary.append({
            'Profile': f'P{k+1}',
            'N': cluster_size,
            'Depression': feature_means_dict['Depression'],
            'Anxiety': feature_means_dict['Anxiety'],
            'Stress': feature_means_dict['Stress'],
            'Burnout': feature_means_dict['Burnout']
        })
        print()

    profile_df = pd.DataFrame(profile_summary)
    print("Profile Summary Table:")
    print(profile_df.to_string(index=False))

    print(f"\n{'='*70}")
    print("Final Model Results Saved:")
    print(f"  - {dataset_name} latent vectors: {latent_vectors_all.shape}")
    print(f"  - {dataset_name} cluster assignments: {cluster_labels_all.shape}")
    print(f"  - {dataset_name} cluster centroids: {cluster_centroids.shape}")
    print(f"  - Profile summary: {len(profile_summary)} profiles")
    print(f"{'='*70}\n")

    # ------------------------------------------------------------------------
    # PROFILE INTERPRETATION
    # ------------------------------------------------------------------------
    # Classify profiles based on symptom levels relative to global thresholds
    # Assign meaningful names (e.g., "Severe Comorbid", "Low Symptom")
    # ------------------------------------------------------------------------
    print("Interpretation of the profiles:")
    print("="*70)


    #It is important to compute global threshold values for each profile based on the entire dataset not just the cluster data
    #This ensures consistency and comparability across different datasets


    global_depression_threshold_high = np.percentile(train_val_data[:, 0], 75)  
    global_depression_threshold_low = np.percentile(train_val_data[:, 0], 25)  
    global_anxiety_threshold_high = np.percentile(train_val_data[:, 1], 75)
    global_anxiety_threshold_low = np.percentile(train_val_data[:, 1], 25)
    global_stress_threshold_high = np.percentile(train_val_data[:, 2], 75)
    global_stress_threshold_low = np.percentile(train_val_data[:, 2], 25)
    global_burnout_threshold_high = np.percentile(train_val_data[:, 3], 75)
    global_burnout_threshold_low = np.percentile(train_val_data[:, 3], 25)

    def interpret_profile(depression, anxiety, stress, burnout):
        """
        Interpret a single profile based on global thresholds
        returns a string description of the profile
        """

        high_symptoms = []
        low_symptoms = []


        # Compare to global thresholds not just cluster centroids
        if depression > global_depression_threshold_high:
            high_symptoms.append("Depression")
        elif depression < global_depression_threshold_low:
            low_symptoms.append("Depression")

        if anxiety > global_anxiety_threshold_high:
            high_symptoms.append("Anxiety")
        elif anxiety < global_anxiety_threshold_low:
            low_symptoms.append("Anxiety")

        if stress > global_stress_threshold_high:
            high_symptoms.append("Stress")
        elif stress < global_stress_threshold_low:
            low_symptoms.append("Stress")

        if burnout > global_burnout_threshold_high:
            high_symptoms.append("Burnout")
        elif burnout < global_burnout_threshold_low:
            low_symptoms.append("Burnout")
    
        if len(high_symptoms) >= 3:
            return "Severe Comorbid Profile", "High levels across multiple dimensions"
        elif "Depression" in high_symptoms and "Anxiety" in high_symptoms:
            return "Depression-Anxiety Comorbidity Profile", "High Depression and Anxiety, typical of internalizing disorders"
        elif "Stress" in high_symptoms and "Burnout" in high_symptoms:
            return "Stress-Burnout Profile", "High Stress and Burnout, typical of work-related distress"
        elif len(high_symptoms) == 1:
            return f"High {high_symptoms[0]} Profile", f"Elevated {high_symptoms[0]} with other symptoms in normal range"
        elif len(low_symptoms) >= 3:
            return "Low Symptom Profile", "Low levels across most dimensions"
        else:
            return "Moderate/Mixed Profile", "Moderate levels across dimensions"

    for i, profile in enumerate(profile_summary):
        profile_name, description = interpret_profile(
            profile['Depression'], 
            profile['Anxiety'], 
            profile['Stress'], 
            profile['Burnout']
        )
        profile_summary[i]['Profile_Name'] = profile_name
        profile_summary[i]['Description'] = description

    # Display interpreted profiles
    print("Interpreted Profiles:")
    interpreted_df = pd.DataFrame(profile_summary)
    print(interpreted_df[['Profile', 'Profile_Name', 'N', 'Depression', 'Anxiety', 'Stress', 'Burnout', 'Description']].to_string(index=False))
    print()

    # ------------------------------------------------------------------------
    # TEST SET EVALUATION
    # ------------------------------------------------------------------------
    # Evaluate model generalization on held-out test data
    # Test set was never used during hyperparameter tuning or training
    # ------------------------------------------------------------------------
    print("Test-Set - 20% of the data was kept aside and used for testing")
    print("="*70)
    print("Test set was held out during hyperparameter tuning - now evaluating final model")
    print("="*70)

    print("Encoding test data with trained autoencoder...")
    test_dataset = TensorDataset(test_tensor.cpu())
    test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False)

    final_model.eval()
    test_latent_vectors = []
    test_reconstructions = []


    with torch.no_grad():
        for batch_data in test_loader:
            data = batch_data[0].to(device)
            latent = final_model.encoder(data)
            reconstructed = final_model(data)
            test_latent_vectors.append(latent.cpu().numpy())
            test_reconstructions.append(reconstructed.cpu().numpy())
    test_latent = np.vstack(test_latent_vectors)
    test_recon = np.vstack(test_reconstructions)


    print(f"  Test latent vectors: {test_latent.shape}")
    print(f"  Test reconstructions: {test_recon.shape}")

    # Compute test reconstruction error
    test_data_np = test_tensor.cpu().numpy()
    test_recon_loss = np.mean((test_data_np - test_recon) ** 2)
    print(f"  Test reconstruction loss (MSE): {test_recon_loss:.6f}")

    # Assign test samples to clusters using trained centroids
    # CRITICAL: Use centroids from train+val, don't retrain K-means
    # This tests if cluster structure generalizes to new data
    print(f"\nAssigning test samples to clusters...")
    from scipy.spatial.distance import cdist
    test_distances = cdist(test_latent, cluster_centroids, metric='euclidean')
    test_cluster_assignments = np.argmin(test_distances, axis=1)

    print(f"  Test cluster assignments: {Counter(test_cluster_assignments)}")

    # Evaluate clustering quality on test set
    # Check if we have at least 2 clusters (required for silhouette score)
    n_unique_test_clusters = len(np.unique(test_cluster_assignments))
    if n_unique_test_clusters < 2:
        print(f"⚠️ ERROR: Test clustering produced only {n_unique_test_clusters} cluster(s). Cannot compute metrics.")
        test_sil_score = -1
        test_ch_score = 0
        test_db_score = float('inf')
    else:
        test_sil_score = silhouette_score(test_latent, test_cluster_assignments)
        test_ch_score = calinski_harabasz_score(test_latent, test_cluster_assignments)
        test_db_score = davies_bouldin_score(test_latent, test_cluster_assignments)

    print(f"\nTest Set Clustering Quality:")
    print(f"  Silhouette Score: {test_sil_score:.6f}")
    print(f"  Calinski-Harabasz: {test_ch_score:.6f}")
    print(f"  Davies-Bouldin: {test_db_score:.6f}")

    # Check for overfitting: Compare train+val vs test performance
    if final_sil_score != 0:
        sil_diff_pct = abs(final_sil_score - test_sil_score) / final_sil_score * 100
    else:
        sil_diff_pct = float('inf')  # Handle edge case where final_sil_score is 0
    if sil_diff_pct < 5:
        print(f"\n✓ Good generalization: Test performance within {sil_diff_pct:.2f}% of train+val")
    elif sil_diff_pct < 10:
        print(f"\n⚠ Moderate generalization gap: Test performance {sil_diff_pct:.2f}% different from train+val")
    else:
        print(f"\n✗ Potential overfitting: Test performance {sil_diff_pct:.2f}% different from train+val")

    print("="*70 + "\n")

    # ------------------------------------------------------------------------
    # VISUALIZATIONS
    # ------------------------------------------------------------------------
    # Create visualizations of latent space and profile characteristics
    # ------------------------------------------------------------------------
    print("Visualizing Latent Space and Profile Characteristics")

    #1. Latent Space Visualization
    print("\n1. Latent Space Viz")

    if best_latent_dim == 2:
        fig, ax = plt.subplots(figsize=(10, 8))
        scatter = ax.scatter(latent_vectors_all[:, 0], latent_vectors_all[:, 1], 
                            c=cluster_labels_all, cmap='viridis', alpha=0.6, s=30)
        ax.scatter(cluster_centroids[:, 0], cluster_centroids[:, 1], c='red', marker='x', 
                  s=300, linewidths=4, label='Cluster Centroids', zorder=5)
        #Add profile labels to centroids

        for i, (x,y) in enumerate(cluster_centroids):
            profile_name = profile_summary[i].get('Profile_Name', f'P{i+1}')
            ax.annotate(profile_name, (x,y), xytext=(5,5), textcoords='offset points',
                       fontsize=10, fontweight='bold', bbox=dict(boxstyle='round,pad=0.3',
                       facecolor='yellow', alpha=0.7))
    
        ax.set_xlabel('Latent Dimension 1', fontsize=12)
        ax.set_ylabel('Latent Dimension 2', fontsize=12)
        ax.set_title('Latent Space Visualization (Colored by Profile)', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.colorbar(scatter, ax=ax, label='Profile')
        plt.tight_layout()
        plt.show()
    
    elif best_latent_dim == 3:
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure(figsize=(12, 8))
        ax = fig.add_subplot(111, projection='3d')
    
        scatter = ax.scatter(latent_vectors_all[:, 0], latent_vectors_all[:, 1], latent_vectors_all[:, 2],
                            c=cluster_labels_all, cmap='viridis', alpha=0.6, s=30)
        ax.scatter(cluster_centroids[:, 0], cluster_centroids[:, 1], cluster_centroids[:, 2],
                  c='red', marker='x', s=300, linewidths=4, label='Centroids')
    
        ax.set_xlabel('Latent Dim 1', fontsize=12)
        ax.set_ylabel('Latent Dim 2', fontsize=12)
        ax.set_zlabel('Latent Dim 3', fontsize=12)
        ax.set_title('Latent Space Visualization (3D)', fontsize=14, fontweight='bold')
        ax.legend()
        plt.colorbar(scatter, ax=ax, label='Profile')
        plt.tight_layout()
        plt.show()

    #2. Profile Characteristics Heatmap
    print("\n2. Profile Characteristics Heatmap:")
    profile_matrix = pd.DataFrame(profile_summary)[['Profile', 'Depression', 'Anxiety', 'Stress', 'Burnout']].set_index('Profile')
    profile_matrix_normalized = (profile_matrix - profile_matrix.min()) / (profile_matrix.max() - profile_matrix.min())

    plt.figure(figsize=(10, 6))
    sns.heatmap(profile_matrix_normalized.T, annot=profile_matrix.T, fmt='.3f', cmap='RdYlGn_r',
               cbar_kws={'label': 'Normalized Symptom Level'}, linewidths=0.5, linecolor='black')
    plt.title('Profile Characteristics Heatmap\n(Normalized Symptom Levels)', fontsize=14, fontweight='bold')
    plt.xlabel('Profile', fontsize=12)
    plt.ylabel('Symptom', fontsize=12)
    plt.tight_layout()
    plt.show()


    #Profile Bar Chart
    print("\n3. Profile Symptom Levels (Bar Chart):")
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(profile_summary))
    width = 0.2

    for i, profile in enumerate(profile_summary):
        ax.bar(x[i] - 1.5*width, profile['Depression'], width, label='Depression' if i == 0 else '', color='#ff6b6b')
        ax.bar(x[i] - 0.5*width, profile['Anxiety'], width, label='Anxiety' if i == 0 else '', color='#4ecdc4')
        ax.bar(x[i] + 0.5*width, profile['Stress'], width, label='Stress' if i == 0 else '', color='#45b7d1')
        ax.bar(x[i] + 1.5*width, profile['Burnout'], width, label='Burnout' if i == 0 else '', color='#f9ca24')

    ax.set_xlabel('Profile', fontsize=12)
    ax.set_ylabel('Symptom Level', fontsize=12)
    ax.set_title('Symptom Levels by Profile', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([p['Profile'] for p in profile_summary])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()


    #Cluster Size Distribution
    print("\n4. Cluster Size Distribution:")

    fig, ax = plt.subplots(figsize=(8, 6))
    cluster_sizes = [p['N'] for p in profile_summary]
    cluster_labels_viz = [p['Profile'] for p in profile_summary]
    colors = plt.cm.viridis(np.linspace(0, 1, len(cluster_sizes)))

    bars = ax.bar(cluster_labels_viz, cluster_sizes, color=colors, edgecolor='black', linewidth=1.5)
    ax.set_xlabel('Profile', fontsize=12)
    ax.set_ylabel('Number of Samples', fontsize=12)
    ax.set_title('Profile Size Distribution', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{int(height)}', ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.show()





    # ------------------------------------------------------------------------
    # H3 VALIDATION: Clinical Utility Testing
    # ------------------------------------------------------------------------
    # Test if profile membership predicts therapy utilization
    # Only available for D1-Swiss dataset
    # ------------------------------------------------------------------------
    if dataset_name == "D1-Swiss" and "PSYT_Therapy_Use" in all_data_df.columns:
        print("\nH3 VALIDATION: Testing Clinical Utility of Profiles")
        print("="*70)
        print("Hypothesis H3: Profile membership is associated with therapy utilization")
        print("Using FULL dataset (train+val+test) for maximum statistical power")
        print("="*70)

        y_therapy = all_data_df["PSYT_Therapy_Use"].values
        train_val_therapy, test_therapy = train_test_split(
            y_therapy,
            test_size=0.2,
            random_state=RANDOM_SEED,
        )
        y_therapy_aligned = np.concatenate([train_val_therapy, test_therapy])
        all_cluster_labels = np.concatenate([cluster_labels_all, test_cluster_assignments])

        assert len(y_therapy_aligned) == len(all_cluster_labels), "Misalignment"
        print(f"\n✓ Data aligned: {len(all_cluster_labels)} samples")
        print(f"  Therapy use rate: {y_therapy_aligned.mean():.2%} ({y_therapy_aligned.sum()}/{len(y_therapy_aligned)})")

        print("\nChi-Square Test for Independence:")
        print("="*70)

        contingency = pd.crosstab(all_cluster_labels, y_therapy_aligned)
        chi2, p, dof, expected = chi2_contingency(contingency)

        print("   Contingency Table:")
        print(contingency)
        print(f"\n   Chi-square statistic: χ² = {chi2:.4f}")
        print(f"   Degrees of freedom: df = {dof}")
        print(f"   p-value: p = {p:.6f}")

        alpha = 0.05
        if p < alpha:
            print(f"\n   ✓ SIGNIFICANT (p < {alpha}): Profile membership IS associated with therapy utilization")
            print("   → H3 VALIDATED: Profiles have clinical utility")
        else:
            print(f"\n   ✗ NOT SIGNIFICANT (p >= {alpha}): No association detected")
            print("   → H3 NOT VALIDATED")

        print("\nCramer V Effect Size:")
        print("="*70)
        n = contingency.values.sum()
        min_dim = min(contingency.shape)
        cramers_v = np.sqrt(chi2 / (n * (min_dim - 1)))
        print(f"   Cramer's V: {cramers_v:.4f}")

        if cramers_v < 0.10:
            effect_size = "negligible"
        elif cramers_v < 0.30:
            effect_size = "small"
        elif cramers_v < 0.50:
            effect_size = "medium"
        else:
            effect_size = "large"
        print(f"   Effect size: {effect_size}")

        print("\n3. Post-Hoc Analysis: Standardized Residuals:")
        print("-"*70)
        print("   (Values > |2| indicate significant deviation from expected)")
        residuals = (contingency.values - expected) / np.sqrt(expected + 1e-10)
        residuals_df = pd.DataFrame(
            residuals,
            index=[f'P{k+1}' for k in range(best_k)],
            columns=['No Therapy', 'Therapy']
        )
        print(residuals_df.round(3))

        print("\n   Significant deviations:")
        for i in range(best_k):
            for j in range(2):
                if abs(residuals[i, j]) > 2:
                    profile_name = profile_summary[i].get('Profile_Name', f'P{i+1}')
                    therapy_status = 'Therapy' if j == 1 else 'No Therapy'
                    direction = 'Higher' if residuals[i, j] > 0 else 'Lower'
                    print(f"      • {profile_name} - {therapy_status}: {direction} than expected (residual = {residuals[i, j]:.2f})")

        print("\n" + "="*70)
        print("H3 VALIDATION SUMMARY:")
        print("="*70)
        print(f"Dataset: {dataset_name} (N={len(all_cluster_labels)})")
        print(f"Chi-square: χ² = {chi2:.4f}, p = {p:.6f}, df = {dof}")
        print(f"Cramér's V = {cramers_v:.4f} ({effect_size} effect)")
        print(f"H3 Status: {' VALIDATED' if p < alpha else '✗ NOT VALIDATED'}")

        if p < alpha:
            print("\nConclusion:")
            print("  Profiles demonstrate clinical utility by predicting therapy utilization.")
            print("  This supports the use of these profiles for targeted mental health interventions.")

        print("="*70 + "\n")
    else:
        print("\nSkipping H3 validation (only available for D1-Swiss with PSYT_Therapy_Use)")

    result = {
        'dataset_name': dataset_name,
        'train_val_data': train_val_data,
        'latent_vectors_all': latent_vectors_all,
        'cluster_labels_all': cluster_labels_all,
        'cluster_centroids': cluster_centroids,
        'best_k': best_k,
        'best_latent_dim': best_latent_dim,
        'final_sil_score': final_sil_score,
        'final_ch_score': final_ch_score,
        'final_db_score': final_db_score,
        'reconstruction_loss': test_recon_loss,  # Store test reconstruction loss for comparison
        'profile_summary': profile_summary,  # Store profile characteristics (mean symptom levels per cluster)
    }
    PIPELINE_RESULTS[dataset_name] = result
    return result



In [ ]:
#Optuna Scout Phase-
#The purpose of this phase is to intelligently explore the hyperparameter space
#Uses 3fold cv, 15 epochs(expedited)

import optuna
from optuna.pruners import MedianPruner

# train_and_validate_model is defined in Cell 3, no import needed

def scout_objective(trial, dataset_name="D1-Swiss"):
    """
    Objective function for Scout hyperparameter optimization.
    """
    #Suggest Hyperparameters
    hidden_size = trial.suggest_int('hidden_size', 2, 12)  # Include 2 for 4-feature datasets
    latent_dim = trial.suggest_int('latent_dim', 2, 4)
    activation = trial.suggest_categorical('activation', ['ReLU', 'Tanh', 'Sigmoid'])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])



    (df, feature_matrix, train_val_data, test_data,
    train_val_tensor, test_tensor, kfold, dataset_path) = prepare_dataset(
        dataset_name
    )

    kfold_scout = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

    activation_map = {
        'ReLU': nn.ReLU,
        'Tanh': nn.Tanh,
        'Sigmoid': nn.Sigmoid
    }
    
    scores = []
    criterion = nn.MSELoss()
    fixed_lr = 1e-3
    for fold_idx, (train_idx, val_idx) in enumerate(kfold_scout.split(train_val_tensor)):
        fold_seed = RANDOM_SEED + fold_idx
        torch.manual_seed(fold_seed)
        np.random.seed(fold_seed)


        train_fold = train_val_tensor[train_idx]
        val_fold = train_val_tensor[val_idx]

        train_dataset = TensorDataset(train_fold.cpu())
        val_dataset = TensorDataset(val_fold.cpu())
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        model = Autoencoder(
            len(FEATURE_COLUMNS),
            hidden_size,
            latent_dim,
            activation_map[activation]

        ).to(device)

        if optimizer_name == 'Adam':
            optimizer = optim.Adam(model.parameters(), lr=fixed_lr)
        else:
            optimizer = optim.SGD(model.parameters(), lr=fixed_lr, momentum=0.9)
        
        # Train and Validate - 15 epochs only
        scout_epochs = 15 

        train_losses, val_losses, optimal_k, best_sil_score, latent_vectors, validation_metrics = \
            train_and_validate_model(model, train_loader, val_loader, optimizer, 
                                    criterion, scout_epochs, device)
        
        fold_score = best_sil_score
        scores.append(fold_score)
        
        # Report intermediate value (enables per-fold pruning)
        # Each fold = 1 step, so with 3-fold CV: 5 steps = 5 ÷ 3 = 1.67 folds
        trial.report(fold_score, step=fold_idx)
        
        # Check if should prune after this fold
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return np.mean(scores) if scores else 0.0
def run_scout_phase(dataset_name="D1-Swiss", n_trials=150, save_path=None):
    print("="*80)
    print(f"SCOUT PHASE: Optuna Optimization on {dataset_name}")
    print("="*80)
    print(f"Configuration:")
    print(f"  - Trials: {n_trials}")
    print(f"  - CV Folds: 3 (fast)")
    print(f"  - Epochs: 15 (fast)")
    print(f"  - Pruning: Enabled (stops bad trials early)")
    print("="*80)

    study = optuna.create_study(
        direction='maximize',
        pruner=MedianPruner(
            n_startup_trials=20,  # Wait for 20 complete trials before pruning
                              # (Ensures stable median baseline, prevents false positives)
            n_warmup_steps=5      # Wait for 5 reporting steps before pruning
                              # With 3-fold CV: 5 steps ÷ 3 folds = 1.67 folds
                              # (Allows 1 complete fold + 2/3 of next fold before pruning)
        )
    )
    start_time = time.time()

    study.optimize(
        lambda trial: scout_objective(trial, dataset_name),
        n_trials=n_trials,
        show_progress_bar=True
    )

    elapsed_time = time.time() - start_time

    print(f"\n✓ Scout phase completed in {elapsed_time/60:.1f} minutes")
    print(f"  Best trial: {study.best_trial.number}")
    print(f"  Best value: {study.best_value:.6f}")
    print(f"  Best params: {study.best_trial.params}")

    top_5_trials = sorted([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE], key=lambda x: x.value if x.value is not None else -1, reverse=True)[:5]
    
    print(f"\nTop 5 Trials:")
    for i, trial in enumerate(top_5_trials, 1):
        print(f"  {i}. Trial {trial.number}: Score={trial.value:.6f}, Params={trial.params}")
    
    # Analyze patterns
    print(f"\nPattern Analysis:")
    hidden_sizes = [t.params['hidden_size'] for t in top_5_trials]
    latent_dims = [t.params['latent_dim'] for t in top_5_trials]
    activations = [t.params['activation'] for t in top_5_trials]
    optimizers = [t.params['optimizer'] for t in top_5_trials]
    
    print(f"  Hidden sizes in top 5: {Counter(hidden_sizes)}")
    print(f"  Latent dims in top 5: {Counter(latent_dims)}")
    print(f"  Activations in top 5: {Counter(activations)}")
    print(f"  Optimizers in top 5: {Counter(optimizers)}")
    
    # Save results
    scout_results = {
        'dataset': dataset_name,
        'n_trials': n_trials,
        'best_trial': study.best_trial.number,
        'best_value': study.best_value,
        'best_params': study.best_trial.params,
        'top_5_trials': [
            {
                'number': t.number,
                'value': t.value,
                'params': t.params
            } for t in top_5_trials
        ],
        'patterns': {
            'hidden_sizes': dict(Counter(hidden_sizes)),
            'latent_dims': dict(Counter(latent_dims)),
            'activations': dict(Counter(activations)),
            'optimizers': dict(Counter(optimizers))
        }
    }
    
    if save_path:
        with open(save_path, 'w') as f:
            json.dump(scout_results, f, indent=2)
        print(f"\n✓ Results saved to {save_path}")
    
    return study, scout_results

In [ ]:
#Extract the narrowed ranges from the optuna exploration study
#Analyse the patterns and trends in the results
#Use these ranges to set the hyperparameters for the next phase

def extract_narrowed_ranges(scout_results):
    """
    Extract narrowed ranges from scout results
    """
    top_5 = scout_results['top_5_trials']

    #Extract ranges from top 5 trials
    hidden_sizes = sorted(set([t['params']['hidden_size'] for t in top_5]))
    latent_dims = sorted(set([t['params']['latent_dim'] for t in top_5]))
    activations = sorted(set([t['params']['activation'] for t in top_5]))
    optimizers = sorted(set([t['params']['optimizer'] for t in top_5]))

            
    hidden_sizes_expanded = sorted(set(
        [max(3, h-1) for h in hidden_sizes] + 
        hidden_sizes + 
        [min(12, h+1) for h in hidden_sizes]
    ))

    if len(activations) == 3:
        act_counts = Counter([t['params']['activation'] for t in top_5])
        activations = [act for act in activations if act_counts[act] >= 2]

    narrowed_params = {
        'hidden_sizes': hidden_sizes_expanded,
        'latent_dims': latent_dims,
        'activations': activations,
        'optimizers': optimizers,
        'epochs': [50, 75, 100]  # Full epochs for focused grid
    }

    print("Narrowed Parameter Ranges for Focused Grid:")
    print(f"  hidden_sizes: {narrowed_params['hidden_sizes']}")
    print(f"  latent_dims: {narrowed_params['latent_dims']}")
    print(f"  activations: {narrowed_params['activations']}")
    print(f"  optimizers: {narrowed_params['optimizers']}")
    print(f"  epochs: {narrowed_params['epochs']}")
    
    grid_size = (len(narrowed_params['hidden_sizes']) * 
                 len(narrowed_params['latent_dims']) * 
                 len(narrowed_params['activations']) * 
                 len(narrowed_params['optimizers']) * 
                 len(narrowed_params['epochs']))
    print(f"\n  Grid size: {grid_size} configs × 10 folds = {grid_size * 10} experiments")
    
    return narrowed_params



In [ ]:
TARGET_DATASET = "D1-Swiss"

from google.colab import drive

drive.mount('/content/drive')

SAVE_DIR = '/content/drive/MyDrive/mental-health-profiling'  # Adjust path as needed

import os 
os.makedirs(SAVE_DIR, exist_ok=True)

print("="*80)
print(f"OPTIMIZATION PIPELINE FOR {TARGET_DATASET}")
print("="*80)


print("="*80)
print("SCOUT PHASE: Optuna Optimization")
print("="*80)

study, scout_results = run_scout_phase(
    dataset_name=TARGET_DATASET,
    n_trials=150,
    save_path=f'{SAVE_DIR}/scout_{TARGET_DATASET.lower().replace("-", "_")}.json'
)


import sqlite3
study_db_path = f'{SAVE_DIR}/scout_{TARGET_DATASET.lower().replace("-", "_")}.db'
study_storage = optuna.storages.InMemoryStorage()
# For simplicity, just save the study as pickle
import pickle
with open(study_db_path.replace('.db', '_study.pkl'), 'wb') as f:
    pickle.dump(study, f)

# Phase 2: Extract narrowed ranges
print("\n" + "="*80)
print("PHASE 2: EXTRACT NARROWED RANGES")
print("="*80)
narrowed_params = extract_narrowed_ranges(scout_results)

with open(f'{SAVE_DIR}/narrowed_params_{TARGET_DATASET.lower().replace("-", "_")}.json', 'w') as f:
    json.dump(narrowed_params, f, indent=2)

# Phase 3: Focused Grid
print("\n" + "="*80)
print("PHASE 3: FOCUSED GRID SEARCH")
print("="*80)
best_result = run_autoencoder_pipeline(
    dataset_name=TARGET_DATASET,
    narrowed_params=narrowed_params
)

# Save final results
with open(f'{SAVE_DIR}/best_config_{TARGET_DATASET.lower().replace("-", "_")}.json', 'w') as f:
    json.dump({
        'dataset': TARGET_DATASET,
        'best_k': best_result['best_k'],
        'best_latent_dim': best_result['best_latent_dim'],
        'final_sil_score': best_result['final_sil_score'],
        'final_ch_score': best_result['final_ch_score'],
        'final_db_score': best_result['final_db_score'],
    }, f, indent=2)

print("\n" + "="*80)
print(f"✓ COMPLETE: {TARGET_DATASET} optimization finished")
print(f"  Results saved to: {SAVE_DIR}")
print("="*80)

In [ ]:
# ============================================================================
# CELL 3: TRAINING AND VALIDATION FUNCTION
# ============================================================================
# Purpose: Train autoencoder and evaluate clustering quality in latent space
# Key feature: Model quality evaluated by clustering metrics, not just
#              reconstruction loss, because goal is finding distinct profiles
# ============================================================================

def train_and_validate_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, device):
    """
    Train and validate the autoencoder model and evaluate reconstruction accuracy and latent quality.
    
    Uses multiple validation methods for K selection with consensus/voting approach:
    - Silhouette score (primary, tiebreaker)
    - Calinski-Harabasz index
    - Davies-Bouldin index
    - Elbow method (WCSS-based knee detection)
    
    K selection: Uses consensus voting - if 2+ methods agree on K, that K is selected.
    If no consensus, falls back to silhouette score (most interpretable).
    
    Returns:
        train_losses: list of training loss values
        val_losses: list of validation loss values
        optimal_k: optimal number of clusters (consensus or silhouette-based)
        best_silhouette_score: best silhouette score achieved
        latent_vectors: latent representations (train+val combined)
        validation_metrics: dict with all K validation metrics and consensus info
    """
    model.train()
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        # Training phase
        epoch_training_loss = 0.0
        for batch_data in train_loader:
            batch_data = batch_data[0].to(device)  # Unpack tuple from DataLoader
            optimizer.zero_grad()
            reconstructed = model(batch_data)
            loss = criterion(reconstructed, batch_data)  # MSE Reconstruction Loss
            loss.backward()
            optimizer.step()
            epoch_training_loss += loss.item()
        avg_train_loss = epoch_training_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        epoch_validation_loss = 0.0
        with torch.no_grad():
            for batch_data in val_loader:
                batch_data = batch_data[0].to(device)
                reconstructed = model(batch_data)
                loss = criterion(reconstructed, batch_data)
                epoch_validation_loss += loss.item()
        avg_val_loss = epoch_validation_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        model.train()

        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")
    
    # ------------------------------------------------------------------------
    # EXTRACT LATENT VECTORS AND EVALUATE CLUSTERING QUALITY
    # ------------------------------------------------------------------------
    # After training, extract latent representations and evaluate clustering
    # ------------------------------------------------------------------------
    model.eval()
    all_latent_vectors = []

    with torch.no_grad():
        # Extract latent vectors from training data
        for batch_data in train_loader:
            data = batch_data[0].to(device)
            latent = model.encoder(data)
            all_latent_vectors.append(latent.cpu().numpy())
        
        # Extract latent vectors from validation data
        for batch_data in val_loader:
            data = batch_data[0].to(device)
            latent = model.encoder(data)
            all_latent_vectors.append(latent.cpu().numpy())
    
    # Combine all latent vectors
    latent_vectors = np.vstack(all_latent_vectors)

    # DIAGNOSTIC: Check if model collapsed
    print(f"\n=== LATENT SPACE DIAGNOSTIC ===")
    print(f"Shape: {latent_vectors.shape}")
    print(f"Mean per dimension: {latent_vectors.mean(axis=0)}")
    print(f"Std per dimension: {latent_vectors.std(axis=0)}")
    print(f"Min per dimension: {latent_vectors.min(axis=0)}")
    print(f"Max per dimension: {latent_vectors.max(axis=0)}")
    print(f"Overall std: {latent_vectors.std():.6f}")
    if latent_vectors.std() < 0.01:
        print("⚠️ MODEL COLLAPSED: All latent vectors are nearly identical!")
        print("   This means the encoder isn't learning useful representations.")
    print("="*70)

    # =========================================================================
    # K Selection via Multiple Validation Methods (Convergent Validity)
    # =========================================================================
    k_range = range(2, 7)  # K = 2, 3, 4, 5, 6
    
    # Initialize metric lists
    silhouette_scores_list = []
    calinski_harabasz_scores = []
    davies_bouldin_scores = []
    wcss_values = []  # Within-cluster sum of squares (for elbow method)
    
    best_silhouette_score = -1
    
    print(f"\nTesting K values: {list(k_range)}")
    print("="*70)
    
    for k in k_range:
        print(f"K={k}: Running K-means...", end=" ", flush=True)
        kmeans = KMeans(n_clusters=k, random_state=RANDOM_SEED, n_init=10)
        cluster_labels = kmeans.fit_predict(latent_vectors)
        print("Done. ", end="", flush=True)
        
        # Check if we have at least 2 clusters (required for silhouette score)
        n_unique_clusters = len(np.unique(cluster_labels))
        if n_unique_clusters < 2:
            print(f"Warning: Only {n_unique_clusters} cluster(s). Skipping metrics.")
            # Assign dummy values that will be ignored
            sil_score = -1
            ch_score = 0
            db_score = float('inf')
        else:
            print("Computing metrics...", end=" ", flush=True)
            # Compute all validation metrics
            sil_score = silhouette_score(latent_vectors, cluster_labels)
            ch_score = calinski_harabasz_score(latent_vectors, cluster_labels)
            db_score = davies_bouldin_score(latent_vectors, cluster_labels)
            print(f"Sil={sil_score:.4f}, CH={ch_score:.2f}, DB={db_score:.4f}")
        wcss = kmeans.inertia_  # Within-cluster sum of squares
        
        # Store metrics
        silhouette_scores_list.append(sil_score)
        calinski_harabasz_scores.append(ch_score)
        davies_bouldin_scores.append(db_score)
        wcss_values.append(wcss)
        
        # Track best silhouette score
        if sil_score > best_silhouette_score:
            best_silhouette_score = sil_score
    
    print("="*70)
    print(f"K-means evaluation complete. Best silhouette score: {best_silhouette_score:.4f}")
    print("\nDetermining optimal K using consensus voting...")
    
    # =========================================================================
    # Consensus/Voting Approach for K Selection
    # =========================================================================
    # Determine optimal K from each method
    optimal_k_silhouette = k_range[np.argmax(silhouette_scores_list)]
    optimal_k_ch = k_range[np.argmax(calinski_harabasz_scores)]  # Highest CH = best
    optimal_k_db = k_range[np.argmin(davies_bouldin_scores)]  # Lowest DB = best
    
    # Elbow Method: Find the "knee" where WCSS decrease rate slows down
    # Compute percentage decrease in WCSS for each K
    wcss_decreases = []
    for i in range(1, len(wcss_values)):
        if wcss_values[i-1] > 0:  # Avoid division by zero
            pct_decrease = ((wcss_values[i-1] - wcss_values[i]) / wcss_values[i-1]) * 100
            wcss_decreases.append(pct_decrease)
        else:
            wcss_decreases.append(0)
    
    # Find elbow: K where decrease rate drops most (knee point)
    # The elbow is where adding more clusters doesn't significantly reduce WCSS
    if len(wcss_decreases) > 0:
        decrease_rates = np.array(wcss_decreases)
        # Method: Find where the decrease rate drops most (elbow detection)
        # Compute the rate of change of decrease rates (second derivative of WCSS)
        if len(decrease_rates) > 1:
            # Rate changes: how much the decrease rate changes between consecutive K
            rate_changes = np.diff(decrease_rates)
            # Elbow is where rate change is most negative (biggest drop in decrease rate)
            # This means: decrease rate was high, then dropped significantly
            elbow_idx = np.argmin(rate_changes) + 1  # +1 because diff reduces length by 1
            # Ensure index is within valid range
            elbow_idx = min(elbow_idx, len(k_range) - 1)
            optimal_k_elbow = k_range[elbow_idx]
        else:
            # Fallback: use K with smallest decrease (conservative)
            min_decrease_idx = np.argmin(wcss_decreases) + 1
            min_decrease_idx = min(min_decrease_idx, len(k_range) - 1)
            optimal_k_elbow = k_range[min_decrease_idx]
    else:
        # Fallback to silhouette if WCSS calculation fails
        optimal_k_elbow = optimal_k_silhouette
    
    # Consensus voting: If 2+ methods agree, use that K
    # Note: Multiple comparisons across 4 metrics and 5 K values (K=2-6) are exploratory
    # We use consensus voting rather to find the optimal K
  
    k_votes = [optimal_k_silhouette, optimal_k_ch, optimal_k_db, optimal_k_elbow]
    k_counts = Counter(k_votes)
    most_common_k, consensus_count = k_counts.most_common(1)[0]
    
    if consensus_count >= 2:
        # Consensus reached: 2+ methods agree
        optimal_k = most_common_k
        consensus_status = f"Consensus: {consensus_count} methods agree on K={optimal_k}"
        consensus_reached = True
    else:
        # No consensus: fallback to silhouette (most interpretable)
        optimal_k = optimal_k_silhouette
        consensus_status = f"No consensus (Sil={optimal_k_silhouette}, CH={optimal_k_ch}, DB={optimal_k_db}, Elbow={optimal_k_elbow}). Using silhouette K={optimal_k}"
        consensus_reached = False
    
    print(f"✓ Optimal K selected: {optimal_k} ({consensus_status})")
    print("="*70)
    
    # Package all validation metrics for analysis
    validation_metrics = {
        'k_values': list(k_range),
        'silhouette_scores': silhouette_scores_list,
        'calinski_harabasz_scores': calinski_harabasz_scores,
        'davies_bouldin_scores': davies_bouldin_scores,
        'wcss_values': wcss_values,
        'wcss_decreases': wcss_decreases if len(wcss_decreases) > 0 else [],
        'optimal_k_silhouette': optimal_k_silhouette,
        'optimal_k_ch': optimal_k_ch,
        'optimal_k_db': optimal_k_db,
        'optimal_k_elbow': optimal_k_elbow,
        'optimal_k_consensus': optimal_k,
        'consensus_reached': consensus_reached,
        'consensus_status': consensus_status,
        'k_votes': k_votes
    }
    
    return train_losses, val_losses, optimal_k, best_silhouette_score, latent_vectors, validation_metrics

In [ ]:
# ============================================================================
# NOTE: This cell previously contained duplicate hyperparameter tuning code
# that was causing NameError because kfold and train_val_tensor are only
# defined inside run_autoencoder_pipeline() function scope.
#
# The hyperparameter tuning code is now properly contained within 
# run_autoencoder_pipeline() function.
#
# To run the pipeline, use:
#   run_autoencoder_pipeline("D1-Swiss")
# ============================================================================
pass

In [ ]:
# =========================================================================
# PCA Comparison: Justify Autoencoder Choice
# =========================================================================

# LEGACY VERSION (kept for reference):
"""
SWISS_DATASET = "D1-Swiss"

if SWISS_DATASET not in PIPELINE_RESULTS:
    print(f"{SWISS_DATASET} not yet processed. Running autoencoder pipeline once for PCA comparison...")
    run_autoencoder_pipeline(SWISS_DATASET)

swiss_results = PIPELINE_RESULTS[SWISS_DATASET]
train_val_data = swiss_results['train_val_data']
latent_vectors_all = swiss_results['latent_vectors_all']
cluster_labels_all = swiss_results['cluster_labels_all']
cluster_centroids = swiss_results['cluster_centroids']
best_k = swiss_results['best_k']
best_latent_dim = swiss_results['best_latent_dim']
final_sil_score = swiss_results['final_sil_score']
final_ch_score = swiss_results['final_ch_score']
final_db_score = swiss_results['final_db_score']

print("METHOD VALIDATION: PCA vs Autoencoder Comparison")
print("="*70)
print("Testing if autoencoder captures nonlinear patterns better than PCA")
print("Testing PCA with all possible dimensions (1-4) to find optimal PCA")
print("="*70)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Prepare data
scaler = StandardScaler()
train_val_scaled = scaler.fit_transform(train_val_data)

# Test PCA with all possible dimensions (1, 2, 3, 4)
print("\n1. Testing PCA with all dimensions:")
pca_results = []

for pca_dim in range(1, 5):  # 1, 2, 3, 4
    # Fit PCA
    pca = PCA(n_components=pca_dim)
    pca_latent = pca.fit_transform(train_val_scaled)
    explained_var = pca.explained_variance_ratio_.sum()

    # Cluster with optimal K (same as autoencoder)
    pca_kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_SEED, n_init=10)
    pca_cluster_labels = pca_kmeans.fit_predict(pca_latent)

    # Evaluate clustering quality
    pca_sil_score = silhouette_score(pca_latent, pca_cluster_labels)
    pca_ch_score = calinski_harabasz_score(pca_latent, pca_cluster_labels)
    pca_db_score = davies_bouldin_score(pca_latent, pca_cluster_labels)

    pca_results.append({
        'dim': pca_dim,
        'explained_var': explained_var,
        'silhouette': pca_sil_score,
        'calinski_harabasz': pca_ch_score,
        'davies_bouldin': pca_db_score
    })

    print(f"   PCA dim={pca_dim}: Sil={pca_sil_score:.6f}, CH={pca_ch_score:.6f}, DB={pca_db_score:.6f}, Var={explained_var:.4f}")

# Find best PCA configuration (by silhouette score)
pca_results_df = pd.DataFrame(pca_results)
best_pca_idx = pca_results_df['silhouette'].idxmax()
best_pca_config = pca_results_df.iloc[best_pca_idx]

print(f"\n   Best PCA: dim={int(best_pca_config['dim'])}, Sil={best_pca_config['silhouette']:.6f}")

# Get best PCA latent vectors for visualization
best_pca_dim = int(best_pca_config['dim'])
best_pca = PCA(n_components=best_pca_dim)
pca_latent_best = best_pca.fit_transform(train_val_scaled)
pca_kmeans_best = KMeans(n_clusters=best_k, random_state=RANDOM_SEED, n_init=10)
pca_cluster_labels_best = pca_kmeans_best.fit_predict(pca_latent_best)
pca_centroids_best = pca_kmeans_best.cluster_centers_

# Autoencoder results (from Cell 7)
print("\n2. Autoencoder (Nonlinear Method):")
print(f"   AE latent dim: {best_latent_dim}")
print(f"   AE latent vectors shape: {latent_vectors_all.shape}")
print(f"   AE Clustering Quality:")
print(f"     Silhouette Score: {final_sil_score:.6f}")
print(f"     Calinski-Harabasz: {final_ch_score:.6f}")
print(f"     Davies-Bouldin: {final_db_score:.6f}")

# Comparison: Best PCA vs Autoencoder
print("\n3. Comparison (Best PCA vs Autoencoder):")
best_pca_sil = best_pca_config['silhouette']
best_pca_ch = best_pca_config['calinski_harabasz']
best_pca_db = best_pca_config['davies_bouldin']

sil_improvement = ((final_sil_score - best_pca_sil) / best_pca_sil) * 100
ch_improvement = ((final_ch_score - best_pca_ch) / best_pca_ch) * 100
db_improvement = ((best_pca_db - final_db_score) / best_pca_db) * 100  # DB: lower is better

print(f"   Silhouette: AE {final_sil_score:.6f} vs Best PCA {best_pca_sil:.6f} ({sil_improvement:+.2f}%)")
print(f"   Calinski-Harabasz: AE {final_ch_score:.6f} vs Best PCA {best_pca_ch:.6f} ({ch_improvement:+.2f}%)")
print(f"   Davies-Bouldin: AE {final_db_score:.6f} vs Best PCA {best_pca_db:.6f} ({db_improvement:+.2f}% improvement)")

# Conclusion
print("\n4. Conclusion:")
if final_sil_score > best_pca_sil:
    print(f"   ✓ Autoencoder achieves {sil_improvement:.2f}% better silhouette score than best PCA")
    print(f"   → Suggests latent structure contains nonlinear patterns")
    print(f"   → Justifies use of autoencoder over linear PCA")
else:
    print(f"   Note: Best PCA (dim={best_pca_dim}) performs similarly ({best_pca_sil:.6f} vs {final_sil_score:.6f})")
    print(f"   → Linear patterns may be sufficient, but AE provides flexibility")

print("="*70 + "\n")

# Visualization: Compare latent spaces
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

if best_pca_dim == 2:
    scatter1 = axes[0].scatter(pca_latent_best[:, 0], pca_latent_best[:, 1], c=pca_cluster_labels_best, 
                              cmap='viridis', alpha=0.6, s=20)
    axes[0].scatter(pca_centroids_best[:, 0], pca_centroids_best[:, 1], c='red', marker='x', s=200, linewidths=3, label='Centroids')
    axes[0].set_xlabel('PC1', fontsize=12)
    axes[0].set_ylabel('PC2', fontsize=12)
    axes[0].set_title(f'Best PCA (dim={best_pca_dim}, Linear)\nSilhouette: {best_pca_sil:.4f}', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    scatter2 = axes[1].scatter(latent_vectors_all[:, 0], latent_vectors_all[:, 1], c=cluster_labels_all,
                              cmap='viridis', alpha=0.6, s=20)
    axes[1].scatter(cluster_centroids[:, 0], cluster_centroids[:, 1], c='red', marker='x', s=200, linewidths=3, label='Centroids')
    axes[1].set_xlabel('Latent Dim 1', fontsize=12)
    axes[1].set_ylabel('Latent Dim 2', fontsize=12)
    axes[1].set_title(f'Autoencoder (dim={best_latent_dim}, Nonlinear)\nSilhouette: {final_sil_score:.4f}', 
                     fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.colorbar(scatter1, ax=axes[0], label='Cluster')
    plt.colorbar(scatter2, ax=axes[1], label='Cluster')
elif best_pca_dim == 3 or best_latent_dim == 3:
    from mpl_toolkits.mplot3d import Axes3D
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')

    if best_pca_dim >= 3:
        ax1.scatter(pca_latent_best[:, 0], pca_latent_best[:, 1], pca_latent_best[:, 2], 
                   c=pca_cluster_labels_best, cmap='viridis', alpha=0.6, s=20)
        ax1.scatter(pca_centroids_best[:, 0], pca_centroids_best[:, 1], pca_centroids_best[:, 2], 
                   c='red', marker='x', s=200, linewidths=3)
    else:
        pca_3d = np.zeros((len(pca_latent_best), 3))
        pca_3d[:, :best_pca_dim] = pca_latent_best
        ax1.scatter(pca_3d[:, 0], pca_3d[:, 1], pca_3d[:, 2], 
                   c=pca_cluster_labels_best, cmap='viridis', alpha=0.6, s=20)
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax1.set_title(f'Best PCA (dim={best_pca_dim})')

    if best_latent_dim >= 3:
        ax2.scatter(latent_vectors_all[:, 0], latent_vectors_all[:, 1], latent_vectors_all[:, 2], 
                   c=cluster_labels_all, cmap='viridis', alpha=0.6, s=20)
        ax2.scatter(cluster_centroids[:, 0], cluster_centroids[:, 1], cluster_centroids[:, 2], 
                   c='red', marker='x', s=200, linewidths=3)
    else:
        ae_3d = np.zeros((len(latent_vectors_all), 3))
        ae_3d[:, :best_latent_dim] = latent_vectors_all
        ax2.scatter(ae_3d[:, 0], ae_3d[:, 1], ae_3d[:, 2], 
                   c=cluster_labels_all, cmap='viridis', alpha=0.6, s=20)
        ax2.scatter(cluster_centroids[:, 0], cluster_centroids[:, 1], np.zeros(len(cluster_centroids)), 
                   c='red', marker='x', s=200, linewidths=3)
    ax2.set_xlabel('Latent Dim 1')
    ax2.set_ylabel('Latent Dim 2')
    ax2.set_zlabel('Latent Dim 3')
    ax2.set_title(f'Autoencoder (dim={best_latent_dim})\nSil: {final_sil_score:.4f} vs PCA: {best_pca_sil:.4f}')
else:
    scatter1 = axes[0].scatter(pca_latent_best[:, 0], np.zeros(len(pca_latent_best)), 
                              c=pca_cluster_labels_best, cmap='viridis', alpha=0.6, s=20)
    axes[0].scatter(pca_centroids_best[:, 0], np.zeros(len(pca_centroids_best)), 
                   c='red', marker='x', s=200, linewidths=3, label='Centroids')
    axes[0].set_xlabel('PC1', fontsize=12)
    axes[0].set_ylabel('(Projected)', fontsize=12)
    axes[0].set_title(f'Best PCA (dim={best_pca_dim})', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    scatter2 = axes[1].scatter(latent_vectors_all[:, 0], np.zeros(len(latent_vectors_all)) if best_latent_dim == 1 else latent_vectors_all[:, 1], 
                              c=cluster_labels_all, cmap='viridis', alpha=0.6, s=20)
    axes[1].scatter(cluster_centroids[:, 0], np.zeros(len(cluster_centroids)) if best_latent_dim == 1 else cluster_centroids[:, 1], 
                   c='red', marker='x', s=200, linewidths=3, label='Centroids')
    axes[1].set_xlabel('Latent Dim 1', fontsize=12)
    axes[1].set_ylabel('Latent Dim 2' if best_latent_dim >= 2 else '(Projected)', fontsize=12)
    axes[1].set_title(f'Autoencoder (dim={best_latent_dim})', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.colorbar(scatter1, ax=axes[0], label='Cluster')
    plt.colorbar(scatter2, ax=axes[1], label='Cluster')

plt.tight_layout()
plt.show()

# Summary table
print("\n5. Summary Table:")
comparison_df = pd.DataFrame({
    'Method': ['Best PCA', 'Autoencoder'],
    'Latent_Dim': [best_pca_dim, best_latent_dim],
    'Silhouette': [best_pca_sil, final_sil_score],
    'Calinski_Harabasz': [best_pca_ch, final_ch_score],
    'Davies_Bouldin': [best_pca_db, final_db_score]
})
print(comparison_df.to_string(index=False))
print()
"""

# ACTIVE VERSION (executes):
SWISS_DATASET = "D1-Swiss"

if SWISS_DATASET not in PIPELINE_RESULTS:
    print(f"{SWISS_DATASET} not yet processed. Running autoencoder pipeline once for PCA comparison...")
    run_autoencoder_pipeline(SWISS_DATASET)

swiss_results = PIPELINE_RESULTS[SWISS_DATASET]
train_val_data = swiss_results['train_val_data']
latent_vectors_all = swiss_results['latent_vectors_all']
cluster_labels_all = swiss_results['cluster_labels_all']
cluster_centroids = swiss_results['cluster_centroids']
best_k = swiss_results['best_k']
best_latent_dim = swiss_results['best_latent_dim']
final_sil_score = swiss_results['final_sil_score']
final_ch_score = swiss_results['final_ch_score']
final_db_score = swiss_results['final_db_score']

print("METHOD VALIDATION: PCA vs Autoencoder Comparison")
print("="*70)
print("Testing if autoencoder captures nonlinear patterns better than PCA")
print("Testing PCA with all possible dimensions (1-4) to find optimal PCA")
print("="*70)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Prepare data
scaler = StandardScaler()
train_val_scaled = scaler.fit_transform(train_val_data)

# Test PCA with all possible dimensions (1, 2, 3, 4)
print("\n1. Testing PCA with all dimensions:")
pca_results = []

for pca_dim in range(1, 5):  # 1, 2, 3, 4
    pca = PCA(n_components=pca_dim)
    pca_latent = pca.fit_transform(train_val_scaled)
    explained_var = pca.explained_variance_ratio_.sum()

    pca_kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_SEED, n_init=10)
    pca_cluster_labels = pca_kmeans.fit_predict(pca_latent)

    pca_sil_score = silhouette_score(pca_latent, pca_cluster_labels)
    pca_ch_score = calinski_harabasz_score(pca_latent, pca_cluster_labels)
    pca_db_score = davies_bouldin_score(pca_latent, pca_cluster_labels)

    pca_results.append({
        'dim': pca_dim,
        'explained_var': explained_var,
        'silhouette': pca_sil_score,
        'calinski_harabasz': pca_ch_score,
        'davies_bouldin': pca_db_score
    })

    print(f"   PCA dim={pca_dim}: Sil={pca_sil_score:.6f}, CH={pca_ch_score:.6f}, DB={pca_db_score:.6f}, Var={explained_var:.4f}")

pca_results_df = pd.DataFrame(pca_results)
best_pca_idx = pca_results_df['silhouette'].idxmax()
best_pca_config = pca_results_df.iloc[best_pca_idx]

print(f"\n   Best PCA: dim={int(best_pca_config['dim'])}, Sil={best_pca_config['silhouette']:.6f}")

best_pca_dim = int(best_pca_config['dim'])
best_pca = PCA(n_components=best_pca_dim)
pca_latent_best = best_pca.fit_transform(train_val_scaled)
pca_kmeans_best = KMeans(n_clusters=best_k, random_state=RANDOM_SEED, n_init=10)
pca_cluster_labels_best = pca_kmeans_best.fit_predict(pca_latent_best)
pca_centroids_best = pca_kmeans_best.cluster_centers_

print("\n2. Autoencoder (Nonlinear Method):")
print(f"   AE latent dim: {best_latent_dim}")
print(f"   AE latent vectors shape: {latent_vectors_all.shape}")
print("   AE Clustering Quality:")
print(f"     Silhouette Score: {final_sil_score:.6f}")
print(f"     Calinski-Harabasz: {final_ch_score:.6f}")
print(f"     Davies-Bouldin: {final_db_score:.6f}")

print("\n3. Comparison (Best PCA vs Autoencoder):")
best_pca_sil = best_pca_config['silhouette']
best_pca_ch = best_pca_config['calinski_harabasz']
best_pca_db = best_pca_config['davies_bouldin']

sil_improvement = ((final_sil_score - best_pca_sil) / best_pca_sil) * 100
ch_improvement = ((final_ch_score - best_pca_ch) / best_pca_ch) * 100
db_improvement = ((best_pca_db - final_db_score) / best_pca_db) * 100

print(f"   Silhouette: AE {final_sil_score:.6f} vs Best PCA {best_pca_sil:.6f} ({sil_improvement:+.2f}%)")
print(f"   Calinski-Harabasz: AE {final_ch_score:.6f} vs Best PCA {best_pca_ch:.6f} ({ch_improvement:+.2f}%)")
print(f"   Davies-Bouldin: AE {final_db_score:.6f} vs Best PCA {best_pca_db:.6f} ({db_improvement:+.2f}% improvement)")

print("\n4. Conclusion:")
if final_sil_score > best_pca_sil:
    print(f"   ✓ Autoencoder achieves {sil_improvement:.2f}% better silhouette score than best PCA")
    print(f"   → Suggests latent structure contains nonlinear patterns")
    print(f"   → Justifies use of autoencoder over linear PCA")
else:
    print(f"   Note: Best PCA (dim={best_pca_dim}) performs similarly ({best_pca_sil:.6f} vs {final_sil_score:.6f})")
    print(f"   → Linear patterns may be sufficient, but AE provides flexibility")

print("="*70 + "\n")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

if best_pca_dim == 2:
    scatter1 = axes[0].scatter(pca_latent_best[:, 0], pca_latent_best[:, 1], c=pca_cluster_labels_best,
                              cmap='viridis', alpha=0.6, s=20)
    axes[0].scatter(pca_centroids_best[:, 0], pca_centroids_best[:, 1], c='red', marker='x', s=200, linewidths=3, label='Centroids')
    axes[0].set_xlabel('PC1', fontsize=12)
    axes[0].set_ylabel('PC2', fontsize=12)
    axes[0].set_title(f'Best PCA (dim={best_pca_dim}, Linear)\nSilhouette: {best_pca_sil:.4f}', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    scatter2 = axes[1].scatter(latent_vectors_all[:, 0], latent_vectors_all[:, 1], c=cluster_labels_all,
                              cmap='viridis', alpha=0.6, s=20)
    axes[1].scatter(cluster_centroids[:, 0], cluster_centroids[:, 1], c='red', marker='x', s=200, linewidths=3, label='Centroids')
    axes[1].set_xlabel('Latent Dim 1', fontsize=12)
    axes[1].set_ylabel('Latent Dim 2', fontsize=12)
    axes[1].set_title(f'Autoencoder (dim={best_latent_dim}, Nonlinear)\nSilhouette: {final_sil_score:.4f}', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.colorbar(scatter1, ax=axes[0], label='Cluster')
    plt.colorbar(scatter2, ax=axes[1], label='Cluster')
elif best_pca_dim == 3 or best_latent_dim == 3:
    from mpl_toolkits.mplot3d import Axes3D
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')

    if best_pca_dim >= 3:
        ax1.scatter(pca_latent_best[:, 0], pca_latent_best[:, 1], pca_latent_best[:, 2],
                   c=pca_cluster_labels_best, cmap='viridis', alpha=0.6, s=20)
        ax1.scatter(pca_centroids_best[:, 0], pca_centroids_best[:, 1], pca_centroids_best[:, 2],
                   c='red', marker='x', s=200, linewidths=3)
    else:
        pca_3d = np.zeros((len(pca_latent_best), 3))
        pca_3d[:, :best_pca_dim] = pca_latent_best
        ax1.scatter(pca_3d[:, 0], pca_3d[:, 1], pca_3d[:, 2],
                   c=pca_cluster_labels_best, cmap='viridis', alpha=0.6, s=20)
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax1.set_title(f'Best PCA (dim={best_pca_dim})')

    if best_latent_dim >= 3:
        ax2.scatter(latent_vectors_all[:, 0], latent_vectors_all[:, 1], latent_vectors_all[:, 2],
                   c=cluster_labels_all, cmap='viridis', alpha=0.6, s=20)
        ax2.scatter(cluster_centroids[:, 0], cluster_centroids[:, 1], cluster_centroids[:, 2],
                   c='red', marker='x', s=200, linewidths=3)
    else:
        ae_3d = np.zeros((len(latent_vectors_all), 3))
        ae_3d[:, :best_latent_dim] = latent_vectors_all
        ax2.scatter(ae_3d[:, 0], ae_3d[:, 1], ae_3d[:, 2],
                   c=cluster_labels_all, cmap='viridis', alpha=0.6, s=20)
        ax2.scatter(cluster_centroids[:, 0], cluster_centroids[:, 1], np.zeros(len(cluster_centroids)),
                   c='red', marker='x', s=200, linewidths=3)
    ax2.set_xlabel('Latent Dim 1')
    ax2.set_ylabel('Latent Dim 2')
    ax2.set_zlabel('Latent Dim 3')
    ax2.set_title(f'Autoencoder (dim={best_latent_dim})\nSil: {final_sil_score:.4f} vs PCA: {best_pca_sil:.4f}')
else:
    scatter1 = axes[0].scatter(pca_latent_best[:, 0], np.zeros(len(pca_latent_best)),
                              c=pca_cluster_labels_best, cmap='viridis', alpha=0.6, s=20)
    axes[0].scatter(pca_centroids_best[:, 0], np.zeros(len(pca_centroids_best)),
                   c='red', marker='x', s=200, linewidths=3, label='Centroids')
    axes[0].set_xlabel('PC1', fontsize=12)
    axes[0].set_ylabel('(Projected)', fontsize=12)
    axes[0].set_title(f'Best PCA (dim={best_pca_dim})', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    scatter2 = axes[1].scatter(latent_vectors_all[:, 0], np.zeros(len(latent_vectors_all)) if best_latent_dim == 1 else latent_vectors_all[:, 1],
                              c=cluster_labels_all, cmap='viridis', alpha=0.6, s=20)
    axes[1].scatter(cluster_centroids[:, 0], np.zeros(len(cluster_centroids)) if best_latent_dim == 1 else cluster_centroids[:, 1],
                   c='red', marker='x', s=200, linewidths=3, label='Centroids')
    axes[1].set_xlabel('Latent Dim 1', fontsize=12)
    axes[1].set_ylabel('Latent Dim 2' if best_latent_dim >= 2 else '(Projected)', fontsize=12)
    axes[1].set_title(f'Autoencoder (dim={best_latent_dim})', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.colorbar(scatter1, ax=axes[0], label='Cluster')
    plt.colorbar(scatter2, ax=axes[1], label='Cluster')

plt.tight_layout()
plt.show()

print("\n5. Summary Table:")
comparison_df = pd.DataFrame({
    'Method': ['Best PCA', 'Autoencoder'],
    'Latent_Dim': [best_pca_dim, best_latent_dim],
    'Silhouette': [best_pca_sil, final_sil_score],
    'Calinski_Harabasz': [best_pca_ch, final_ch_score],
    'Davies_Bouldin': [best_pca_db, final_db_score]
})
print(comparison_df.to_string(index=False))
print()



In [ ]:
# =========================================================================
# REPLICATION TESTING: H1/H2 Hypotheses (D1-Swiss as Reference)
# =========================================================================
# Only matches latent_dim (required for comparison), K can differ
# Reuses existing run_autoencoder_pipeline() function
# =========================================================================

# Additional imports (already imported in Cell 0, but included for clarity)
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

# Configuration constants
EXTERNAL_DATASETS = ["D2-Cultural", "D3-Academic", "D4-Tech"]
REPLICATION_THRESHOLD_H1 = 0.70
REPLICATION_THRESHOLD_H2 = 0.50

print("="*80)
print("REPLICATION TESTING: Testing Profile Generalizability")
print("="*80)
print("Hypothesis H1: Universal profiles (r > 0.70)")
print("Hypothesis H2: Context-specific profiles (r < 0.50)")
print("="*80)
print("Reference: D1-Swiss (only latent_dim must match for comparison)")
print("="*80)

# ========================================================================
# STEP 1: PROCESS D1-SWISS (REFERENCE DATASET)
# ========================================================================
print("\nStep 1: Processing D1-Swiss (reference dataset)...")
print("-"*80)

if "D1-Swiss" not in PIPELINE_RESULTS:
    print("Running pipeline on D1-Swiss...")
    run_autoencoder_pipeline("D1-Swiss")
else:
    print("D1-Swiss already processed ✓")

d1_results = PIPELINE_RESULTS["D1-Swiss"]
d1_latent_dim = d1_results['best_latent_dim']
d1_k = d1_results['best_k']
d1_centroids = d1_results['cluster_centroids']

print(f"\nD1-Swiss Reference Configuration:")
print(f"  Optimal K: {d1_k}")
print(f"  Latent dimension: {d1_latent_dim}D")
print(f"  Centroids shape: {d1_centroids.shape}")

# ========================================================================
# STEP 2: PROCESS EXTERNAL DATASETS AND RETUNE IF NEEDED
# ========================================================================
print(f"\nStep 2: Processing external datasets and retuning to {d1_latent_dim}D if needed...")
print("-"*80)

standardized_results = {}

for ext_dataset in EXTERNAL_DATASETS:
    print(f"\nProcessing {ext_dataset}...")
    
    # Run pipeline if not already processed
    if ext_dataset not in PIPELINE_RESULTS:
        print(f"  Running pipeline on {ext_dataset}...")
        run_autoencoder_pipeline(ext_dataset)
    
    ext_result = PIPELINE_RESULTS[ext_dataset]
    ext_latent_dim = ext_result['best_latent_dim']
    
    print(f"  Original latent dimensions: {ext_latent_dim}D")
    
    if ext_latent_dim == d1_latent_dim:
        print(f"  ✓ Latent dimension matches D1-Swiss ({d1_latent_dim}D)")
        standardized_results[ext_dataset] = ext_result.copy()
    else:
        print(f"  ⚠ Dimension mismatch: {ext_latent_dim}D ≠ {d1_latent_dim}D")
        print(f"  → Retuning to match D1-Swiss latent_dim ({d1_latent_dim}D)...")
        print(f"    (K can differ - only latent_dim needs to match for comparison)")
        
        retrained_result = run_autoencoder_pipeline(
            ext_dataset, 
            force_latent_dim=d1_latent_dim
        )
        
        original_recon_loss = ext_result.get('reconstruction_loss', None)
        
        if original_recon_loss is not None:
            retrained_recon_loss = retrained_result.get('reconstruction_loss', None)
            
            if retrained_recon_loss is not None:
                if original_recon_loss != 0:
                    info_loss_pct = (
                        (retrained_recon_loss - original_recon_loss) / original_recon_loss
                    ) * 100
                    
                    print(f"\n  Information Loss Analysis ({ext_latent_dim}D → {d1_latent_dim}D):")
                    print(f"    Original recon loss ({ext_latent_dim}D): {original_recon_loss:.6f}")
                    print(f"    Retuned recon loss ({d1_latent_dim}D): {retrained_recon_loss:.6f}")
                    print(f"    Increase: {info_loss_pct:+.2f}% (higher = more information lost)")
                    
                    retrained_result['info_loss_pct'] = info_loss_pct
                    retrained_result['original_recon_loss'] = original_recon_loss
                else:
                    print(f"\n  Information Loss Analysis ({ext_latent_dim}D → {d1_latent_dim}D):")
                    print(f"    Original recon loss ({ext_latent_dim}D): {original_recon_loss:.6f}")
                    print(f"    Retuned recon loss ({d1_latent_dim}D): {retrained_recon_loss:.6f}")
                    print(f"    Note: Cannot calculate percentage change (original loss is 0)")
        
        standardized_results[ext_dataset] = retrained_result

# ========================================================================
# STEP 3: COMPARE PROFILE CHARACTERISTICS ACROSS DATASETS
# ========================================================================
# PRIMARY: Symptom pattern comparison (natural K - preserves structure)
# SECONDARY: Forced K matching (robustness check - standardized comparison)
# ========================================================================
print(f"\n{'='*80}")
print(f"Step 3: Comparing profile characteristics across datasets")
print(f"{'='*80}")
print("METHODOLOGICAL APPROACH:")
print("  PRIMARY: Symptom pattern comparison (natural K)")
print("    - Preserves each dataset's optimal K (natural structure)")
print("    - Compares actual symptom patterns (Depression, Anxiety, Stress, Burnout)")
print("    - Uses Hungarian algorithm to match best subset when K differs")
print("    - Tests: 'Are symptom patterns similar across datasets?'")
print("  SECONDARY: Forced K matching (robustness check)")
print("    - Re-clusters external datasets to match D1's K")
print("    - Standardized comparison with same structure")
print("    - Tests: 'Can we find matching profiles if we force same structure?'")
print("\nNOTE: Different optimal K values are interpreted as evidence for H2")
print("      (context-specificity), but this is integrated into interpretation,")
print("      not a separate comparison method.")
print(f"{'='*80}")

# Extract D1-Swiss profile characteristics
d1_profile_summary = d1_results.get('profile_summary', [])
if not d1_profile_summary:
    print("⚠ WARNING: D1-Swiss profile_summary not found. Falling back to centroid comparison.")
    d1_profile_summary = None

replication_results = []

for ext_dataset in EXTERNAL_DATASETS:
    print(f"\n{'='*80}")
    print(f"Comparing {ext_dataset} profiles to D1-Swiss")
    print(f"{'='*80}")
    
    ext_result = standardized_results[ext_dataset]
    ext_k = ext_result['best_k']
    ext_profile_summary = ext_result.get('profile_summary', [])
    
    print(f"  D1-Swiss K: {d1_k}")
    print(f"  {ext_dataset} K: {ext_k}")
    
    # Check if K differs (preliminary evidence for interpretation)
    k_different = (ext_k != d1_k)
    if k_different:
        print(f"     Note: Different optimal K ({d1_k} vs {ext_k}) suggests context-specificity")
    
    # ====================================================================
    # PRIMARY METHOD: Symptom Pattern Comparison (Natural K)
    # ====================================================================
    if d1_profile_summary and ext_profile_summary:
        print(f"\n  PRIMARY: Symptom Pattern Comparison (Natural K)")
        print(f"  {'-'*70}")
        print(f"    Testing: 'Are symptom patterns similar across datasets?'")
        if ext_k != d1_k:
            print(f"    Note: Comparing {min(d1_k, ext_k)} of {max(d1_k, ext_k)} profiles")
            print(f"    → Preserving natural structure (not forcing K matching)")
            print(f"    → Extra profiles in larger dataset indicate additional structure")
        
        # Extract symptom vectors (4 features: Depression, Anxiety, Stress, Burnout)
        d1_symptom_matrix = np.array([
            [p['Depression'], p['Anxiety'], p['Stress'], p['Burnout']]
            for p in d1_profile_summary
        ])
        
        ext_symptom_matrix = np.array([
            [p['Depression'], p['Anxiety'], p['Stress'], p['Burnout']]
            for p in ext_profile_summary
        ])
        
        print(f"    D1-Swiss symptom matrix: {d1_symptom_matrix.shape} (K={d1_k} × 4 features)")
        print(f"    {ext_dataset} symptom matrix: {ext_symptom_matrix.shape} (K={ext_k} × 4 features)")
        
        # Match profiles using Hungarian Algorithm on symptom patterns
        # Distance = Euclidean distance in 4D symptom space
        symptom_distance_matrix = cdist(d1_symptom_matrix, ext_symptom_matrix, metric='euclidean')
        row_indices, col_indices = linear_sum_assignment(symptom_distance_matrix)
        ext_symptom_matched = ext_symptom_matrix[col_indices]
        
        # Calculate feature-wise correlations
        feature_correlations = []
        feature_p_values = []
        feature_names = ['Depression', 'Anxiety', 'Stress', 'Burnout']
        
        for feat_idx, feat_name in enumerate(feature_names):
            d1_feat = d1_symptom_matrix[:, feat_idx]
            ext_feat = ext_symptom_matched[:, feat_idx]
            # Use minimum K for correlation (smaller sample size)
            min_k = min(d1_k, ext_k)
            correlation, p_value = pearsonr(d1_feat[:min_k], ext_feat[:min_k])
            feature_correlations.append(correlation)
            feature_p_values.append(p_value)
        
        # Calculate overall correlation on matched symptom profiles
        # Flatten both matrices and compute correlation
        d1_symptom_flat = d1_symptom_matrix.flatten()
        ext_symptom_flat = ext_symptom_matched.flatten()
        # Use minimum length to handle different K values
        min_len = min(len(d1_symptom_flat), len(ext_symptom_flat))
        overall_symptom_corr, overall_symptom_p = pearsonr(
            d1_symptom_flat[:min_len], 
            ext_symptom_flat[:min_len]
        )
        
        avg_feature_corr = np.mean(feature_correlations)
        
        print(f"\n    Feature-wise correlations (n={min(d1_k, ext_k)} profiles):")
        for feat_name, corr, p_val in zip(feature_names, feature_correlations, feature_p_values):
            print(f"      {feat_name}: r={corr:.4f} (p={p_val:.6f})")
        print(f"    Average feature correlation: {avg_feature_corr:.4f}")
        print(f"    Overall symptom pattern correlation: {overall_symptom_corr:.4f} (p={overall_symptom_p:.6f}, n={min_len})")
        
        # Test hypotheses using symptom pattern correlation (PRIMARY)
        h1_supported_primary = overall_symptom_corr > REPLICATION_THRESHOLD_H1
        h2_supported_primary = overall_symptom_corr < REPLICATION_THRESHOLD_H2
        
        print(f"\n    Symptom Pattern Correlation (PRIMARY TEST):")
        print(f"      Correlation: r={overall_symptom_corr:.4f}")
        print(f"      H1 threshold (r > {REPLICATION_THRESHOLD_H1}): {'Met' if h1_supported_primary else 'Not Met'}")
        print(f"      H2 threshold (r < {REPLICATION_THRESHOLD_H2}): {'Met' if h2_supported_primary else 'Not Met'}")
        
        # Combined interpretation: K difference informs interpretation, symptom correlation is PRIMARY
        print(f"\n    INTERPRETATION:")
        if k_different:
            print(f"      Preliminary: Different K ({d1_k} vs {ext_k}) → Suggests context-specificity")
            if h2_supported_primary:
                print(f"      PRIMARY: Low symptom correlation (r={overall_symptom_corr:.4f}) → CONFIRMS H2")
                print(f"      → CONCLUSION: CONTEXT-SPECIFIC (both K and patterns differ)")
            elif h1_supported_primary:
                print(f"      PRIMARY: High symptom correlation (r={overall_symptom_corr:.4f}) → CONFLICTS with K evidence")
                print(f"      → CONCLUSION: CONTEXT-SPECIFIC (K differs, but some patterns similar)")
                print(f"      → Note: Different K is strong evidence for context-specificity")
            else:
                print(f"      PRIMARY: Moderate correlation (r={overall_symptom_corr:.4f}) → NEUTRAL")
                print(f"      → CONCLUSION: CONTEXT-SPECIFIC (K difference is strong evidence)")
        else:
            # Same K, so symptom correlation is the deciding factor
            print(f"      Preliminary: Same K ({d1_k}) → Neutral")
            if h1_supported_primary:
                print(f"      PRIMARY: High symptom correlation (r={overall_symptom_corr:.4f}) → SUPPORTS H1")
                print(f"      → CONCLUSION: UNIVERSAL (same structure, similar patterns)")
            elif h2_supported_primary:
                print(f"      PRIMARY: Low symptom correlation (r={overall_symptom_corr:.4f}) → SUPPORTS H2")
                print(f"      → CONCLUSION: CONTEXT-SPECIFIC (same structure, different patterns)")
            else:
                print(f"      PRIMARY: Moderate correlation (r={overall_symptom_corr:.4f}) → NEUTRAL")
                print(f"      → CONCLUSION: MIXED (same structure, moderate pattern similarity)")
        
        primary_correlation = overall_symptom_corr
        primary_p_value = overall_symptom_p
        h1_supported_primary = h1_supported_primary
        h2_supported_primary = h2_supported_primary
    else:
        print(f"\n   Profile characteristics not available, skipping symptom pattern comparison")
        primary_correlation = None
        primary_p_value = None
        h1_supported_primary = False
        h2_supported_primary = False
    
    # ====================================================================
    # SECONDARY METHOD: Forced K Matching (Robustness Check)
    # ====================================================================
    print(f"\n  SECONDARY: Forced K Matching (Robustness Check)")
    print(f"  {'-'*70}")
    
    ext_centroids = ext_result['cluster_centroids']
    
    # Re-cluster if K differs
    if ext_k != d1_k:
        print(f"    Re-clustering {ext_dataset} with K={d1_k} to match D1-Swiss...")
        ext_latent_vectors = ext_result['latent_vectors_all']
        ext_kmeans = KMeans(n_clusters=d1_k, random_state=RANDOM_SEED, n_init=10)
        ext_cluster_labels = ext_kmeans.fit_predict(ext_latent_vectors)
        ext_centroids = ext_kmeans.cluster_centers_
        ext_k_forced = d1_k
    else:
        ext_k_forced = ext_k
    
    # Match centroids using Hungarian Algorithm
    d1_centroids = d1_results['cluster_centroids']
    distance_matrix = cdist(d1_centroids, ext_centroids, metric='euclidean')
    row_indices, col_indices = linear_sum_assignment(distance_matrix)
    ext_centroids_matched = ext_centroids[col_indices]
    
    # Calculate dimension-wise correlations
    dim_correlations = []
    dim_p_values = []
    
    for dim in range(d1_latent_dim):
        d1_dim = d1_centroids[:, dim]
        ext_dim = ext_centroids_matched[:, dim]
        correlation, p_value = pearsonr(d1_dim, ext_dim)
        dim_correlations.append(correlation)
        dim_p_values.append(p_value)
    
    avg_correlation = np.mean(dim_correlations)
    
    # Overall correlation on centroids
    d1_flat = d1_centroids.flatten()
    ext_flat = ext_centroids_matched.flatten()
    overall_centroid_corr, overall_centroid_p = pearsonr(d1_flat, ext_flat)
    
    print(f"    Centroid correlation (forced K={d1_k}): {overall_centroid_corr:.4f} (p={overall_centroid_p:.6f})")
    print(f"    ⚠ Note: Forced K matching may mask true differences")
    
    h1_supported_secondary = overall_centroid_corr > REPLICATION_THRESHOLD_H1
    h2_supported_secondary = overall_centroid_corr < REPLICATION_THRESHOLD_H2
    
    # Store results
    replication_results.append({
        'dataset': ext_dataset,
        'd1_k': d1_k,
        'ext_k': ext_result['best_k'],  # Original K, not forced
        'k_different': k_different,  # Preliminary evidence: Different K suggests H2
        'primary_correlation': primary_correlation if primary_correlation is not None else None,
        'primary_p_value': primary_p_value if primary_p_value is not None else None,
        'avg_feature_correlation': avg_feature_corr if d1_profile_summary and ext_profile_summary else None,
        'h1_supported_primary': h1_supported_primary if primary_correlation is not None else False,
        'h2_supported_primary': h2_supported_primary if primary_correlation is not None else False,
        'secondary_correlation': overall_centroid_corr,  # Robustness check
        'secondary_p_value': overall_centroid_p,
        'avg_dim_correlation': avg_correlation,
        'h1_supported_secondary': h1_supported_secondary,
        'h2_supported_secondary': h2_supported_secondary,
        'info_loss_pct': ext_result.get('info_loss_pct', None),
    })
    
    # ====================================================================
    # VISUALIZATION
    # ====================================================================
    if d1_latent_dim == 2:
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # D1-Swiss centroids
        axes[0].scatter(
            d1_centroids[:, 0],
            d1_centroids[:, 1],
            c=range(d1_k),
            cmap='viridis',
            s=200,
            edgecolors='black',
            linewidth=2,
            marker='o',
            label='D1-Swiss'
        )
        
        for i, (x, y) in enumerate(d1_centroids):
            axes[0].annotate(
                f'P{i+1}',
                (x, y),
                xytext=(5, 5),
                textcoords='offset points',
                fontweight='bold'
            )
        
        axes[0].set_xlabel('Latent Dim 1', fontsize=12)
        axes[0].set_ylabel('Latent Dim 2', fontsize=12)
        axes[0].set_title('D1-Swiss Centroids (Reference)', fontsize=14, fontweight='bold')
        axes[0].grid(True, alpha=0.3)
        axes[0].legend()
        
        # External dataset centroids
        axes[1].scatter(
            ext_centroids_matched[:, 0],
            ext_centroids_matched[:, 1],
            c=range(ext_k),
            cmap='viridis',
            s=200,
            edgecolors='red',
            linewidth=2,
            marker='s',
            label=ext_dataset
        )
        
        for i, (x, y) in enumerate(ext_centroids_matched):
            axes[1].annotate(
                f'P{i+1}',
                (x, y),
                xytext=(5, 5),
                textcoords='offset points',
                fontweight='bold'
            )
        
        axes[1].set_xlabel('Latent Dim 1', fontsize=12)
        axes[1].set_ylabel('Latent Dim 2', fontsize=12)
        axes[1].set_title(
            f'{ext_dataset} Centroids (Matched)\nr = {overall_centroid_corr:.4f}',
            fontsize=14,
            fontweight='bold'
        )
        axes[1].grid(True, alpha=0.3)
        axes[1].legend()
        
        plt.tight_layout()
        plt.show()
    
    elif d1_latent_dim == 3:
        fig = plt.figure(figsize=(14, 6))
        
        ax1 = fig.add_subplot(121, projection='3d')
        ax1.scatter(
            d1_centroids[:, 0],
            d1_centroids[:, 1],
            d1_centroids[:, 2],
            c=range(d1_k),
            cmap='viridis',
            s=200,
            edgecolors='black',
            linewidth=2
        )
        ax1.set_xlabel('Latent Dim 1')
        ax1.set_ylabel('Latent Dim 2')
        ax1.set_zlabel('Latent Dim 3')
        ax1.set_title('D1-Swiss Centroids (Reference)')
        
        ax2 = fig.add_subplot(122, projection='3d')
        ax2.scatter(
            ext_centroids_matched[:, 0],
            ext_centroids_matched[:, 1],
            ext_centroids_matched[:, 2],
            c=range(ext_k),
            cmap='viridis',
            s=200,
            edgecolors='red',
            linewidth=2
        )
        ax2.set_xlabel('Latent Dim 1')
        ax2.set_ylabel('Latent Dim 2')
        ax2.set_zlabel('Latent Dim 3')
        ax2.set_title(f'{ext_dataset} Centroids\nr = {overall_centroid_corr:.4f}')
        
        plt.tight_layout()
        plt.show()

# ========================================================================
# SUMMARY AND CONCLUSIONS
# ========================================================================
print("="*80)
print("REPLICATION TESTING SUMMARY")
print("="*80)
print("\nMETHODOLOGICAL APPROACH:")
print("  PRIMARY: Symptom pattern comparison (natural K)")
print("    - Preserves each dataset's optimal K (natural structure)")
print("    - Compares actual symptom patterns (Depression, Anxiety, Stress, Burnout)")
print("    - Uses Hungarian algorithm to match best subset when K differs")
print("    - Tests: 'Are symptom patterns similar across datasets?'")
print("  SECONDARY: Forced K matching (robustness check)")
print("    - Re-clusters external datasets to match D1's K")
print("    - Standardized comparison with same structure")
print("    - Compares centroids in latent space")
print("    - Tests: 'Can we find matching profiles if we force same structure?'")
print("\nINTERPRETATION NOTES:")
print("  - Different optimal K values are interpreted as evidence for H2")
print("    (context-specificity), but this is integrated into interpretation,")
print("    not a separate comparison method")
print("  - Correlation computed on profiles (n=K, typically 2-6)")
print("  - P-values should be interpreted with caution due to small sample size")
print("  - Correlation coefficient (r) is the primary metric for hypothesis testing")
print("="*80)

replication_df = pd.DataFrame(replication_results)

# Display K comparison (preliminary evidence)
display_cols_k = [
    'dataset',
    'd1_k',
    'ext_k',
    'k_different'
]

print("\nPRELIMINARY: Optimal K Comparison (Informs Interpretation)")
print("="*80)
k_display = replication_df[display_cols_k].copy()
k_display.columns = ['Dataset', 'D1 K', 'Ext K', 'K Different (H2 Evidence)']
k_display['K Different (H2 Evidence)'] = k_display['K Different (H2 Evidence)'].map({True: '✓ YES (H2)', False: '✗ NO (Neutral)'})
print(k_display.to_string(index=False))

# Display PRIMARY results (symptom patterns)
display_cols_primary = [
    'dataset',
    'primary_correlation',
    'avg_feature_correlation',
    'h1_supported_primary',
    'h2_supported_primary'
]

print("\n\nPRIMARY: Symptom Pattern Comparison (Natural K)")
print("="*80)
primary_display = replication_df[display_cols_primary].copy()
primary_display.columns = ['Dataset', 'Symptom Pattern r', 'Avg Feature r', 'H1 Supported', 'H2 Supported']
print(primary_display.to_string(index=False))

# Display SECONDARY results (forced K)
display_cols_secondary = [
    'dataset',
    'secondary_correlation',
    'avg_dim_correlation',
    'h1_supported_secondary',
    'h2_supported_secondary'
]

print("\n\nSECONDARY: Forced K Matching (Robustness Check)")
print("="*80)
secondary_display = replication_df[display_cols_secondary].copy()
secondary_display.columns = ['Dataset', 'Centroid r', 'Avg Dim r', 'H1 Supported', 'H2 Supported']
print(secondary_display.to_string(index=False))

# Count evidence
k_different_count = sum(replication_df['k_different'] == True)  # Preliminary evidence
h1_count_primary = sum(replication_df['h1_supported_primary'] == True)  # PRIMARY
h2_count_primary = sum(replication_df['h2_supported_primary'] == True)  # PRIMARY
h1_count_secondary = sum(replication_df['h1_supported_secondary'] == True)  # SECONDARY
h2_count_secondary = sum(replication_df['h2_supported_secondary'] == True)  # SECONDARY
total_tested = len(replication_df)

print(f"\n{'='*80}")
print("OVERALL CONCLUSION")
print(f"{'='*80}")
print(f"Reference: D1-Swiss ({d1_latent_dim}D, K={d1_k})")
print(f"All datasets standardized to: {d1_latent_dim}D latent space")
print(f"Datasets tested: {total_tested}")
print(f"\nPRELIMINARY (Optimal K Comparison):")
print(f"  Different K (H2 evidence): {k_different_count}/{total_tested} datasets")
print(f"  Same K (Neutral): {total_tested - k_different_count}/{total_tested} datasets")
print(f"\nPRIMARY (Symptom Pattern Comparison - Natural K):")
print(f"  H1 (Universal) supported: {h1_count_primary}/{total_tested}")
print(f"  H2 (Context-Specific) supported: {h2_count_primary}/{total_tested}")
print(f"\nSECONDARY (Forced K Matching - Robustness Check):")
print(f"  H1 (Universal) supported: {h1_count_secondary}/{total_tested}")
print(f"  H2 (Context-Specific) supported: {h2_count_secondary}/{total_tested}")

# Final conclusion based on PRIMARY method (symptom pattern comparison)
# K difference is integrated as preliminary evidence
if k_different_count >= 2:
    # Different K is strong preliminary evidence for H2
    print(f"\n{'='*80}")
    print("✓ H2 VALIDATED: Profiles are CONTEXT-SPECIFIC")
    print(f"{'='*80}")
    print(f"  PRELIMINARY: {k_different_count}/{total_tested} datasets have different optimal K")
    print(f"    → Strong evidence that profiles are context-specific")
    print(f"    → Different populations have different profile structures")
    if h2_count_primary >= 2:
        print(f"  PRIMARY: {h2_count_primary}/{total_tested} datasets show low symptom pattern correlation")
        print(f"    → Confirms context-specificity (both structure and patterns differ)")
    elif h1_count_primary >= 1:
        print(f"  PRIMARY: Some datasets show high symptom pattern correlation despite different K")
        print(f"    → Note: Different K is strong evidence for context-specificity")
    print(f"  → Conclusion: Mental health profiles vary by population/culture/context")
elif k_different_count == 1:
    print(f"\n{'='*80}")
    print("→ MIXED EVIDENCE: Partial support for context-specificity")
    print(f"{'='*80}")
    print(f"  PRELIMINARY: 1/{total_tested} dataset has different optimal K")
    print(f"    → Suggests some context-specificity, but not universal")
    if h2_count_primary >= 2:
        print(f"  PRIMARY: {h2_count_primary}/{total_tested} datasets show low symptom pattern correlation")
        print(f"    → Additional evidence for context-specificity")
    print(f"  → Conclusion: Profiles may be context-specific in some populations but not others")
else:
    # All have same K, so PRIMARY (symptom correlation) is deciding factor
    print(f"\n{'='*80}")
    if h1_count_primary >= 2:
        print("✓ H1 VALIDATED: Profiles appear to be UNIVERSAL")
        print(f"{'='*80}")
        print(f"  PRELIMINARY: All datasets have same optimal K ({d1_k}) - Neutral")
        print(f"  PRIMARY: {h1_count_primary}/{total_tested} datasets show high symptom pattern correlation")
        print(f"    → Conclusion: Mental health profiles are consistent across populations")
        print(f"    → Similar mean levels of Depression, Anxiety, Stress, Burnout across datasets")
    elif h2_count_primary >= 2:
        print("✓ H2 VALIDATED: Profiles appear to be CONTEXT-SPECIFIC")
        print(f"{'='*80}")
        print(f"  PRELIMINARY: All datasets have same optimal K ({d1_k}) - Neutral")
        print(f"  PRIMARY: {h2_count_primary}/{total_tested} datasets show low symptom pattern correlation")
        print(f"    → Conclusion: Same structure but different symptom patterns")
        print(f"    → Profiles vary by population/culture/context")
    else:
        print("→ MIXED RESULTS: Moderate replication")
        print(f"{'='*80}")
        print(f"  PRELIMINARY: All datasets have same optimal K ({d1_k}) - Neutral")
        print(f"  PRIMARY: Symptom patterns show moderate correlation (0.50 ≤ r ≤ 0.70)")
        print(f"    → May depend on specific dataset characteristics")
        print(f"    → Consider dataset-specific factors (culture, demographics, measurement tools)")

print(f"{'='*80}\n")

In [ ]:
# =========================================================================
# OPTIONAL: Process All Datasets Automatically
# =========================================================================
# Uncomment the function call below to process all datasets through
# the autoencoder pipeline automatically
# =========================================================================

def run_all_datasets():
    """
    Process all datasets through the autoencoder pipeline.
    
    This function iterates through all datasets defined in DATASETS
    and runs the complete pipeline (hyperparameter tuning + profile extraction)
    for each one.
    """
    for dataset_name in DATASETS:
        print(f"\n{'#'*80}\nProcessing {dataset_name}\n{'#'*80}\n")
        run_autoencoder_pipeline(dataset_name)

# Uncomment the line below to run:
# run_all_datasets()



In [ ]:
#h1/h2 = replication testing
#Purpose : Identify which are universal vs context-specific
#Two pronged approach : Primary(Natural K) vs Secondary(Forced K)
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
from sklearn.cluster import KMeans

ALL_DATASETS = ["D1-Swiss", "D2-Cultural", "D3-Academic", "D4-Tech"]
REPLICATION_THRESHOLD_H1 = 0.70  # r > 0.70 = Universal
REPLICATION_THRESHOLD_H2 = 0.50  # r < 0.50 = Context-specific

print("="*80)
print("PAIRWISE REPLICATION TESTING")
print("="*80)
print("Comparing all datasets to each other to identify universal vs context-specific profiles")
print("Two-pronged approach: Natural K (PRIMARY) + Forced K (SECONDARY)")
print("="*80)

#Process the datasets
print("\nProcessing datasets...")
print("="*80)

for dataset_name in ALL_DATASETS:
    if dataset_name not in PIPELINE_RESULTS:
        print(f"  Processing {dataset_name}...")
        run_autoencoder_pipeline(dataset_name)
    else:
        print(f"  {dataset_name} already processed ✓")

#Standardize to common latent dim
print("\nStandardizing to common latent dimension...")
print("="*80)

#Get all unique latent dimensions from results
#use swiss dataset as reference
swiss_latent_dim = PIPELINE_RESULTS["D1-Swiss"]["best_latent_dim"]
print(f"Swiss dataset has latent dimension: {swiss_latent_dim}")

standardized_results = {}

for dataset_name in ALL_DATASETS:
    result = PIPELINE_RESULTS[dataset_name]

    if result['best_latent_dim'] != swiss_latent_dim:
        print(f"Standardizing {dataset_name} to {swiss_latent_dim}D...")
        result = run_autoencoder_pipeline(dataset_name, force_latent_dim=swiss_latent_dim)
    standardized_results[dataset_name] = result
    print(f"  ✓ {dataset_name}: K={result['best_k']}, latent_dim={result['best_latent_dim']}D")

#Pairwise comparison for all the pairs
print("\n" + "="*80)
print(" Pairwise Profile Comparisons (All Unique Pairs)")
print("="*80)
print("Comparing symptom patterns (Depression, Anxiety, Stress, Burnout)")
print("="*80)

def compare_profiles(dataset1_name, dataset2_name, results_dict, force_k=None):
    """
    Compare profiles between two datasets using symptom patterns
    
    Args:
        dataset1_name (str): Name of the first dataset
        dataset2_name (str): Name of the second dataset
        results_dict (dict): Dictionary containing all dataset results
        force_k (int, optional): If provided, re-cluster both datasets to this K
                                 If None, uses natural K from each dataset
    """
    result1 = results_dict[dataset1_name]
    result2 = results_dict[dataset2_name]

    #If force k - re-cluster both datasets
    if force_k is not None:
        from sklearn.cluster import KMeans


         # Re-cluster dataset1
        latent1 = result1['latent_vectors_all']
        train_val_data1 = result1['train_val_data']
        kmeans1 = KMeans(n_clusters=force_k, random_state=RANDOM_SEED, n_init=10)
        labels1 = kmeans1.fit_predict(latent1)
        
        # Re-cluster dataset2
        latent2 = result2['latent_vectors_all']
        train_val_data2 = result2['train_val_data']
        kmeans2 = KMeans(n_clusters=force_k, random_state=RANDOM_SEED, n_init=10)
        labels2 = kmeans2.fit_predict(latent2)

        #Recompute the profile summaries with the new Forced K

        profile1 = []
        profile2 = []

        for k in range(force_k):
            # Dataset 1
            mask1 = labels1 == k
            if np.sum(mask1) > 0:
                cluster1 = train_val_data1[mask1]
                profile1.append({
                    'Depression': cluster1[:, 0].mean(),
                    'Anxiety': cluster1[:, 1].mean(),
                    'Stress': cluster1[:, 2].mean(),
                    'Burnout': cluster1[:, 3].mean()
                })
            
            # Dataset 2
            mask2 = labels2 == k
            if np.sum(mask2) > 0:
                cluster2 = train_val_data2[mask2]
                profile2.append({
                    'Depression': cluster2[:, 0].mean(),
                    'Anxiety': cluster2[:, 1].mean(),
                    'Stress': cluster2[:, 2].mean(),
                    'Burnout': cluster2[:, 3].mean()
                })
    else:
        # Use natural K (original approach)
        profile1 = result1.get('profile_summary', [])
        profile2 = result2.get('profile_summary', [])
        
    if not profile1 or not profile2:
        return None, None, False, False
    
    symptom_matrix1 = np.array([
        [p['Depression'], p['Anxiety'], p['Stress'], p['Burnout']]
        for p in profile1
    ])

    symptom_matrix2 = np.array([
        [p['Depression'], p['Anxiety'], p['Stress'], p['Burnout']]
        for p in profile2
    ])

    #match the profiles using the job assignment algorithm(hungarian algorithm)

    distance_matrix = cdist(symptom_matrix1, symptom_matrix2, metric='euclidean')
    row_ind, col_ind = linear_sum_assignment(distance_matrix)
    symptom_matrix2_matched = symptom_matrix2[col_ind]

    flat1 = symptom_matrix1.flatten()
    flat2 = symptom_matrix2_matched.flatten()

    #Handle different k (if not forced k)

    if force_k is None:
        min_len = min(len(flat1), len(flat2))
        if min_len < 4:  # Need at least 1 profile (4 symptoms)
            return None, None, False, False
        correlation, p_value = pearsonr(flat1[:min_len], flat2[:min_len])
    else:
        # Same K, can use full arrays
        if len(flat1) != len(flat2):
            return None, None, False, False
        correlation, p_value = pearsonr(flat1, flat2)
    # Test hypotheses
    h1_supported = correlation > REPLICATION_THRESHOLD_H1
    h2_supported = correlation < REPLICATION_THRESHOLD_H2

    return correlation, p_value, h1_supported, h2_supported

pairwise_results = []
for i in range(len(ALL_DATASETS)):
    for j in range(i+1, len(ALL_DATASETS)):
        dataset1_name = ALL_DATASETS[i]
        dataset2_name = ALL_DATASETS[j]
        print(f"\nComparing {dataset1_name} and {dataset2_name}...")

        result1 = standardized_results[dataset1_name]
        result2 = standardized_results[dataset2_name]

        k1 = result1['best_k']
        k2 = result2['best_k']
        k_different = k1 != k2    

        correlation_natural, p_value_natural, h1_supported_natural, h2_supported_natural = compare_profiles(
            dataset1_name, dataset2_name, standardized_results, force_k=None
        )

        #Secondary - forced k

        reference_k = standardized_results["D1-Swiss"]["best_k"]
        if k_different:
            correlation_forced, p_forced, h1_forced, h2_forced = compare_profiles(
                dataset1_name, dataset2_name, standardized_results, force_k=reference_k
            )
        else:
            # Same K, forced = natural
            correlation_forced = correlation_natural
            p_forced = p_value_natural
            h1_forced = h1_supported_natural
            h2_forced = h2_supported_natural

        print(f"    K: {k1} vs {k2} ({'Different' if k_different else 'Same'})")

        if correlation_natural is not None:
            print(f"    PRIMARY (Natural K):")
            print(f"      Correlation: r={correlation_natural:.4f} (p={p_value_natural:.6f})")
            print(f"      H1 (Universal): {'✔️' if h1_supported_natural else '✗'}")
            print(f"      H2 (Context-specific): {'✔️' if h2_supported_natural else '✗'}")            

        else:
            print(f"    PRIMARY (Natural K): No valid profiles found")
        

        if correlation_forced is not None:
            print(f"Secondary (Forced K):")
            print(f"      Correlation: r={correlation_forced:.4f} (p={p_forced:.6f})")
            print(f"      H1 (Universal): {'✔️' if h1_forced else '✗'}")
            print(f"      H2 (Context-specific): {'✔️' if h2_forced else '✗'}")
        else:
            print(f"Secondary (Forced K): No valid profiles found")
        
        pairwise_results.append({
            'dataset1': dataset1_name,
            'dataset2': dataset2_name,
            'k1': k1,
            'k2': k2,
            'k_different': k_different,
            'correlation_natural': correlation_natural,
            'correlation_forced': correlation_forced,
            'p_value_natural': p_value_natural,
            'p_value_forced': p_forced,
            'h1_supported_natural': h1_supported_natural if correlation_natural is not None else False,
            'h2_supported_natural': h2_supported_natural if correlation_natural is not None else False,
            'h1_supported_forced': h1_forced if correlation_forced is not None else False,
            'h2_supported_forced': h2_forced if correlation_forced is not None else False,
            'correlation': correlation_natural,
            'p_value': p_value_natural,
            'h1_supported': h1_supported_natural if correlation_natural is not None else False,
            'h2_supported': h2_supported_natural if correlation_natural is not None else False,
        })



            


    #Create correlation matrix and analysis

print("\nCreating correlation matrix and analysis...")
print("="*80)

pairwise_df = pd.DataFrame(pairwise_results)

correlation_matrix_natural = np.ones((len(ALL_DATASETS), len(ALL_DATASETS))) * np.nan
correlation_matrix_forced = np.ones((len(ALL_DATASETS), len(ALL_DATASETS))) * np.nan

for index, row in pairwise_df.iterrows():
    i = ALL_DATASETS.index(row['dataset1'])
    j = ALL_DATASETS.index(row['dataset2'])
    correlation_natural = row['correlation_natural'] if row['correlation_natural'] is not None else np.nan
    correlation_forced = row['correlation_forced'] if row['correlation_forced'] is not None else np.nan

    correlation_matrix_natural[i, j] = correlation_natural
    correlation_matrix_forced[i, j] = correlation_forced

    correlation_matrix_natural[j, i] = correlation_natural
    correlation_matrix_forced[j, i] = correlation_forced

correlation_df_natural = pd.DataFrame(
    correlation_matrix_natural,
    index=ALL_DATASETS,
    columns=ALL_DATASETS
)

correlation_df_forced = pd.DataFrame(
    correlation_matrix_forced,
    index=ALL_DATASETS,
    columns=ALL_DATASETS
)

print("\nPRIMARY: Natural K Correlation Matrix (Preserves Original Structure):")
print(correlation_df_natural.to_string())

print("\nSECONDARY: Forced K Correlation Matrix (K=" + str(standardized_results["D1-Swiss"]['best_k']) + ", D1-Swiss Reference):")
print(correlation_df_forced.to_string())

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Natural K
sns.heatmap(correlation_df_natural, annot=True, fmt='.3f', cmap='RdYlGn', center=0.6,
            vmin=0, vmax=1, square=True, linewidths=0.5, cbar_kws={'label': 'Correlation'}, ax=axes[0])
axes[0].set_title('PRIMARY: Natural K (Preserves Original Structure)\n(Higher = More Similar Profiles)', 
          fontsize=14, fontweight='bold')

# Forced K
sns.heatmap(correlation_df_forced, annot=True, fmt='.3f', cmap='RdYlGn', center=0.6,
            vmin=0, vmax=1, square=True, linewidths=0.5, cbar_kws={'label': 'Correlation'}, ax=axes[1])
axes[1].set_title('SECONDARY: Forced K=' + str(standardized_results["D1-Swiss"]['best_k']) + ' (D1-Swiss Reference)\n(Higher = More Similar Profiles)', 
          fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

#Identify universal and context-specific profiles
print("\nIdentifying Universal and Context-Specific Profiles:")
print("="*80)

# Create a dictionary to store the results
dataset_average_correlations = {}

for dataset in ALL_DATASETS:
    correlations = []
    for other_dataset in ALL_DATASETS:
        if dataset != other_dataset:
            match = pairwise_df[
                ((pairwise_df['dataset1'] == dataset) & (pairwise_df['dataset2'] == other_dataset)) |
                ((pairwise_df['dataset1'] == other_dataset) & (pairwise_df['dataset2'] == dataset))
            ]
            if len(match) > 0 and match.iloc[0]['correlation_natural'] is not None:
                correlations.append(match.iloc[0]['correlation_natural'])
    
    if correlations:
        avg_correlation = np.mean(correlations)
        dataset_average_correlations[dataset] = avg_correlation

        if avg_correlation > REPLICATION_THRESHOLD_H1:
            classification = "Universal (high similarity to all others)"
        elif avg_correlation < REPLICATION_THRESHOLD_H2:
            classification = "Context-specific (low similarity to all others)"
        else:
            classification = "Mixed (intermediate similarity)"
        
        print(f"\n{dataset}: {classification}")
        print(f"  Average Correlation: {avg_correlation:.4f}")
        print(f"  Correlations with others: {[f'{c:.3f}' for c in correlations]}")

#Final Print 
print("\nFinal Classification:")
print("="*80)

universal_datasets = [d for d, corr in dataset_average_correlations.items() 
                      if corr > REPLICATION_THRESHOLD_H1]
context_specific_datasets = [d for d, corr in dataset_average_correlations.items() 
                            if corr < REPLICATION_THRESHOLD_H2]
mixed_datasets = [d for d, corr in dataset_average_correlations.items() 
                 if REPLICATION_THRESHOLD_H2 <= corr <= REPLICATION_THRESHOLD_H1]

print(f"\nUniversal Profiles (r > {REPLICATION_THRESHOLD_H1}): {universal_datasets}")
print(f"Context-Specific Profiles (r < {REPLICATION_THRESHOLD_H2}): {context_specific_datasets}")
print(f"Mixed Profiles ({REPLICATION_THRESHOLD_H2} ≤ r ≤ {REPLICATION_THRESHOLD_H1}): {mixed_datasets}")

h1_pairs_natural = sum(pairwise_df['h1_supported_natural'])
h2_pairs_natural = sum(pairwise_df['h2_supported_natural'])
h1_pairs_forced = sum(pairwise_df['h1_supported_forced'])
h2_pairs_forced = sum(pairwise_df['h2_supported_forced'])
total_pairs = len(pairwise_df)

print(f"\n{'='*80}")
print("PRIMARY: Natural K Comparisons (Preserves Original Structure)")
print(f"{'='*80}")
print(f"  Total pairs: {total_pairs}")
print(f"  H1 (Universal) supported: {h1_pairs_natural}/{total_pairs} pairs")
print(f"  H2 (Context-specific) supported: {h2_pairs_natural}/{total_pairs} pairs")


print(f"\n{'='*80}")
print("SECONDARY: Forced K Comparisons (K=" + str(standardized_results["D1-Swiss"]['best_k']) + ", D1-Swiss Reference)")
print(f"{'='*80}")
print(f"  Total pairs: {total_pairs}")
print(f"  H1 (Universal) supported: {h1_pairs_forced}/{total_pairs} pairs")
print(f"  H2 (Context-specific) supported: {h2_pairs_forced}/{total_pairs} pairs")

print(f"\n{'='*80}")
print("OVERALL CONCLUSION (Based on PRIMARY: Natural K)")
print(f"{'='*80}")



if h1_pairs_natural >= total_pairs * 0.5:
    print(f"\n→ H1 VALIDATED: Profiles appear UNIVERSAL across most datasets")
    print(f"  → {len(universal_datasets)}/{len(ALL_DATASETS)} datasets show high average correlation")
    print(f"  → Mental health profiles are consistent across populations")
elif h2_pairs_natural >= total_pairs * 0.5:
    print(f"\n→ H2 VALIDATED: Profiles appear CONTEXT-SPECIFIC across most datasets")
    print(f"  → {len(context_specific_datasets)}/{len(ALL_DATASETS)} datasets show low average correlation")
    print(f"  → Mental health profiles vary by population/culture/context")
else:
    print(f"\n→ MIXED EVIDENCE: Profiles show partial replication")
    print(f"  → Some datasets universal, some context-specific")
    print(f"  → May depend on specific dataset characteristics")

print("="*80)





